In [ ]:
#r "nuget: Plotly.NET, 4.0.0"
#r "nuget: Plotly.NET.Interactive, 4.0.0"
#r "nuget: FSharp.Stats"

open Plotly.NET


# Growth curve

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/fslaborg/FSharp.Stats/gh-pages?urlpath=/tree/home/jovyan/GrowthCurve.ipynb)
[![Notebook](https://fslab.org/FSharp.Stats/img/badge-notebook.svg)](https://fslab.org/FSharp.Stats/GrowthCurve.ipynb)

**Summary:** this tutorial demonstrates variou way to model growth curves, a commong task in any (micro)biological lab

### Table of contents

* [Modelling](#Modelling)

* [Manual phase selection](#Manual-phase-selection)

* [Gompertz model](#Gompertz-model)

* [Generation time calculation](#Generation-time-calculation)

* [Other models](#Other-models)

  * [Richards curve](#Richards-curve)
  
  * [Weibull](#Weibull)
  
  * [Janoschek](#Janoschek)
  
  * [Exponential](#Exponential)
  
  * [Verhulst](#Verhulst)
  
  * [Morgan-Mercer-Flodin](#Morgan-Mercer-Flodin)
  
  * [von Bertalanffy](#von-Bertalanffy)
  

* [Comparison between all models](Comparison-between-all-models)

  * [Fit function](#Fit-function)
  
  * [Generation time](#Generation-time)
  

* [Model examples](#Model-examples)

## Modelling

Growth and other physiological parameters like size/weight/length can be modeled as function of time.
Several growth curve models have been proposed. Some of them are covered in this documentation.

For growth curve analysis the cell count data must be considered in log space. The exponential phase (log phase) then becomes linear.
After modeling, all growth parameters (maximal cell count, lag phase duration, and growth rate) can be derived from the model, so there is no
need for manual labelling of separate growth phases.



In [ ]:
</br>


![Data model](img/growthCurve.png)



In [ ]:
</br>


If specific parameters should be constrained to the users choice (like upper or lower asymptote), a constrained version of the
Levenberg-Marquardt solver can be used (`LevenbergMarquardtConstrained`)! Accordingly, minimal and maximal parameter vectors must be provided.



In [3]:
open System
open FSharp.Stats
open FSharp.Stats.Fitting.NonLinearRegression
open FSharp.Stats.Fitting.LinearRegression

let time = [|0. .. 0.5 .. 8.|]

let cellCount =
    [|
    17000000.;16500000.;11000000.;14000000.;27000000.;40000000.;120000000.;
    300000000.;450000000.;1200000000.;2700000000.;5000000000.;
    8000000000.;11700000000.;12000000000.;13000000000.;12800000000.
    |]

let cellCountLn = cellCount |> Array.map log

open Plotly.NET

let chartOrig = 
    Chart.Point(time,cellCount)
    |> Chart.withTraceInfo "original data"
    |> Chart.withTemplate ChartTemplates.lightMirrored
    |> Chart.withYAxisStyle "cells/ml"

let chartLog =
    Chart.Point(time,cellCountLn)
    |> Chart.withTraceInfo "log count"
    |> Chart.withTemplate ChartTemplates.lightMirrored
    |> Chart.withXAxisStyle "time (h)"
    |> Chart.withYAxisStyle "ln(cells/ml)"
    
let growthChart = 
    [chartOrig;chartLog] |> Chart.Grid(2,1)


In [ ]:
growthChart


<!-- Plotly chart will be drawn inside this DIV -->

## Manual phase selection

If growth phases are labelled manually, the exponential phase can be fitted with a regression line.

To determine the generation time, it is necessary to find the time interval it takes to double the count data.
When a log<sub>2</sub> transform is used, a doubling of the original counts is achieved, when the log value moves 1 unit.
Keeping that in mind, the slope can be used to calculate the time it takes for the log<sub>2</sub> data to increase 1 unit.

* slope * generation time = 1
  

* generation time = 1/slope
  

If a different log transform was used, the correction factor for the numerator is log<sub>x</sub>(2).



In [5]:
// The exponential phase was defined to be between time point 1.5 and 5.5.
let logPhaseX = vector time.[3..11]
let logPhaseY = vector cellCountLn.[3..11]

// regression coefficients as [intercept;slope]
let regressionCoeffs =
    OrdinaryLeastSquares.Linear.Univariable.coefficient logPhaseX logPhaseY


Here is a pre-evaluated version (to save time during the build process, as the solver takes quite some time.)



In [6]:
// The generation time is calculated by dividing log_x 2. by the regression line slope. 
// The log transform must match the used data transform. 
let slope = regressionCoeffs.[1]
let generationTime = log(2.) / slope


let fittedValues = 
    let f = OrdinaryLeastSquares.Linear.Univariable.fit (vector [|14.03859475; 1.515073487|])
    logPhaseX |> Seq.map (fun x -> x,f x)

let chartLinearRegression =
    [
    Chart.Point(time,cellCountLn)   |> Chart.withTraceInfo "log counts"
    Chart.Line(fittedValues)        |> Chart.withTraceInfo "fit"
    ]
    |> Chart.combine
    |> Chart.withTemplate ChartTemplates.lightMirrored
    |> Chart.withXAxisStyle "time (h)"
    |> Chart.withYAxisStyle "ln(cells/ml)"


In [ ]:
chartLinearRegression


<!-- Plotly chart will be drawn inside this DIV -->

In [8]:
let generationTimeManual = sprintf "The generation time (manual selection) is: %.1f min" ((log(2.)/1.5150)* 60.)

generationTimeManual


The generation time (manual selection) is: 27.5 min

## Gompertz model

In the following example the four parameter gompertz function is applied to cell count data (Gibson et al., 1988).

The Gompertz model is fitted using the Levenberg Marquardt solver. Initial parameters are estimated from the original data.
The expected generation time has to be approximated as initial guess.

For parameter interpretation the applied log transform is important and must be provided.
If other parameters than cell count (e.g. size or length) should be analyzed, use `id` as value transform.

Gompertz parameters:

* A: lower asymptote
  

* B: relative growth rate (approximated by generation time consideration)
  

* C: upper asymptote - lower asymptote
  

* M: time point of inflection (maximal growth rate)
  



In [9]:
// The Levenberg Marquardt algorithm identifies the parameters that leads to the best fit 
// of the gompertz models to the count data. The solver must be provided with initial parameters
// that are estimated in the following:
let solverOptions (xData :float []) (yDataLog :float []) expectedGenerationTime (usedLogTransform: float -> float) =
    // lower asymptote
    let a = Seq.min yDataLog
    // upper asymptote - lower asymptote (y range)
    let c = (Seq.max yDataLog) - a
    // relative growth rate
    let b = usedLogTransform 2. * Math.E / (expectedGenerationTime * c)
    // time point of inflection (in gompertz model at f(x)=36% of the y range)
    let m = 
        let yAtInflection = a + c * 0.36
        Seq.zip xData yDataLog
        |> Seq.minBy (fun (xValue,yValue) ->
            Math.Abs (yValue - yAtInflection)
        )
        |> fst
    createSolverOption 0.001 0.001 10000 [|a;b;c;m|]

// By solving the nonlinear fitting problem, the optimal model parameters are determined
let gompertzParams =
    LevenbergMarquardt.estimatedParams
        Table.GrowthModels.gompertz
        (solverOptions time cellCountLn 1. log)
        0.1
        10.
        time
        cellCountLn

let fittingFunction = 
    Table.GrowthModels.gompertz.GetFunctionValue gompertzParams

    
let fittedValuesGompertz =
    // The parameter were determined locally for saving time during build processes
    //let f = Table.GrowthModels.gompertz.GetFunctionValue (vector [|16.46850199; 0.7014917539; 7.274139441; 3.3947717|])
    [time.[0] .. 0.1 .. Seq.last time]
    |> Seq.map (fun x -> 
        x,fittingFunction x
        ) 
    |> Chart.Line
    |> Chart.withTraceInfo "gompertz"

let fittedChartGompertz = 
    [
        Chart.Point(time,cellCountLn)
        |> Chart.withTraceInfo "log count"
        fittedValuesGompertz
    ]
    |> Chart.combine
    |> Chart.withTemplate ChartTemplates.lightMirrored
    |> Chart.withXAxisStyle "time (h)"
    |> Chart.withYAxisStyle "ln(cells/ml)"


In [ ]:
fittedChartGompertz


<!-- Plotly chart will be drawn inside this DIV -->

## Generation time calculation

The generation time can be calculated by dividing log(2) by the slope of the inflection point. The used log transform must
match the used log transform applied to the count data.

The four parameter Gompertz model allows the determination of generation times from its parameters (Gibson et al., 1988).



In [11]:
let generationtime (parametervector:vector) (logTransform:float -> float) =
    logTransform 2. * Math.E / (parametervector.[1] * parametervector.[2])

let lag (parametervector:vector) =
    (parametervector.[3] - 1.) / parametervector.[1]

let g = sprintf "The generation time (Gompertz) is: %.1f min" (60. * (generationtime gompertzParams log))
let l = sprintf "The lag phase duration is %.2f h" (lag gompertzParams)


"The generation time (Gompertz) is: 22.2 min"

"The lag phase duration is 3.41 h"

## Other models

In the following other growth models are applied to the given data set:

* Richards
  

* Weibull
  

* Janoschek
  

* Exponential
  

* Verhulst
  

* Morgan-Mercer-Flodin
  

* von Bertalanffy
  

To determine the generation time, the slope at the inflection point must be calculated.
As explained above, the generation time can be calculated by: logx(2)/(slope at inflection) where x is the used
log transform.

[Choose a appropriate growth model according to your needs.](http://www.pisces-conservation.com/growthhelp/index.html)

For an overview please scroll down to see a combined diagram of all growth models.

### Richards curve

Parameters:

* l: upper asymptote
  

* k: growth rate
  

* y: inflection point (x value)
  

* d: influences the inflection point on the y axis
  



In [ ]:
let richardsParams =
    LevenbergMarquardt.estimatedParams
        Table.GrowthModels.richards
        (createSolverOption 0.001 0.001 10000 [|23.;1.;3.4;2.|])
        0.1
        10.
        time
        cellCountLn

let fittingFunctionRichards = 
    Table.GrowthModels.richards.GetFunctionValue richardsParams


Here is a pre-evaluated version (to save time during the build process, as the solver takes quite some time.)



In [14]:
let generationtimeRichards (richardParameters:vector) =
    let l = richardParameters.[0]
    let k = richardParameters.[1]
    let y = richardParameters.[2] //x value of inflection point
    let d = richardParameters.[3]
    let gradientFunctionRichards t =
        (k*l*((d-1.)*Math.Exp(-k*(t-y))+1.)**(1./(1.-d)))/(Math.Exp(k*(t-y))+d-1.)
    let maximalSlope =
        gradientFunctionRichards y
    log(2.) / maximalSlope

let fittedValuesRichards =
    // The parameter were determined locally for saving time during build processes
    let f =  Table.GrowthModels.richards.GetFunctionValue (vector [|23.25211263; 7.053516315; 5.646889803; 111.0132522|])
    [time.[0] .. 0.1 .. Seq.last time]
    |> Seq.map (fun x -> 
        x,f x
        ) 
    |> Chart.Line

let fittedChartRichards = 
    fittedValuesRichards
    |> Chart.withTemplate ChartTemplates.lightMirrored
    |> Chart.withXAxisStyle "time (h)"
    |> Chart.withYAxisStyle "ln(cells/ml)"
    |> Chart.withTraceInfo "richards"

let fittedChartRichardsS = 
    [
        Chart.Point(time,cellCountLn)
        |> Chart.withTraceInfo "log count"
        fittedValuesRichards
    ]
    |> Chart.combine
    |> Chart.withTemplate ChartTemplates.lightMirrored
    |> Chart.withXAxisStyle "time (h)"
    |> Chart.withYAxisStyle "ln(cells/ml)"
    |> Chart.withTitle "richards"

let generationRichards = sprintf "The generation time (Richards) is: %.1f min" (generationtimeRichards (vector [|23.25211263; 7.053516315; 5.646889803; 111.0132522|]) * 60.)


In [ ]:
fittedChartRichardsS


<!-- Plotly chart will be drawn inside this DIV -->

"The generation time (Richards) is: 29.4 min"

### Weibull

Parameters:

* b: lower asymptote
  

* l: upper asymptote
  

* k: growth rate
  

* d: influences the inflection point position
  



In [ ]:
let weibullParams =
    LevenbergMarquardt.estimatedParams
        Table.GrowthModels.weibull
        (createSolverOption 0.001 0.001 10000 [|15.;25.;1.;5.|])
        0.1
        10.
        time
        cellCountLn

let fittingFunctionWeibull = 
    Table.GrowthModels.weibull.GetFunctionValue weibullParams


Here is a pre-evaluated version (to save time during the build process, as the solver takes quite some time.)



In [17]:
let generationtimeWeibull (weibullParameters:vector) =
    let b = weibullParameters.[0]
    let l = weibullParameters.[1]
    let k = weibullParameters.[2]
    let d = weibullParameters.[3]
    let gradientFunctionWeibull t =
        (d*(l-b)*(k*t)**d*Math.Exp(-((k*t)**d)))/t
    let inflectionPointXValue =
        (1./k)*((d-1.)/d)**(1./d)
    let maximalSlope =
        gradientFunctionWeibull inflectionPointXValue
    log(2.) / maximalSlope

let fittedValuesWeibull =
    // The parameter were determined locally for saving time during build processes
    let f =  Table.GrowthModels.weibull.GetFunctionValue (vector [|16.40632433; 23.35537293; 0.2277752116; 2.900806071|])
    [time.[0] .. 0.1 .. Seq.last time]
    |> Seq.map (fun x -> 
        x,f x
        ) 
    |> Chart.Line

let fittedChartWeibull = 
    fittedValuesWeibull
    |> Chart.withTemplate ChartTemplates.lightMirrored
    |> Chart.withXAxisStyle "time (h)"
    |> Chart.withYAxisStyle "ln(cells/ml)"
    |> Chart.withTraceInfo "weibull"

let fittedChartWeibullS = 
    [
        Chart.Point(time,cellCountLn)
        |> Chart.withTraceInfo "log count"
        fittedValuesWeibull
    ]
    |> Chart.combine
    |> Chart.withTemplate ChartTemplates.lightMirrored
    |> Chart.withXAxisStyle "time (h)"
    |> Chart.withYAxisStyle "ln(cells/ml)"
    |> Chart.withTitle "weibull"

let generationWeibull = 
    sprintf "The generation time (Weibull) is: %.1f min" (generationtimeWeibull (vector [|16.40632433; 23.35537293; 0.2277752116; 2.900806071|]) * 60.)   


In [ ]:
fittedChartWeibullS


<!-- Plotly chart will be drawn inside this DIV -->

"The generation time (Weibull) is: 23.0 min"

### Janoschek

Parameters:

* b: lower asymptote
  

* l: upper asymptote
  

* k: growth rate
  

* d: influences the inflection point position on the x axis
  



In [ ]:
let janoschekParams =
    LevenbergMarquardt.estimatedParams
        Table.GrowthModels.janoschek
        (createSolverOption 0.001 0.001 10000 [|15.;25.;1.;5.|])
        0.1
        10.
        time
        cellCountLn

let fittingFunctionJanoschek = 
    Table.GrowthModels.janoschek.GetFunctionValue janoschekParams


Here is a pre-evaluated version (to save time during the build process, as the solver takes quite some time.)



In [20]:
let generationtimeJanoschek (janoschekParameters:vector) =
    let b = janoschekParameters.[0]
    let l = janoschekParameters.[1]
    let k = janoschekParameters.[2]
    let d = janoschekParameters.[3]
    let gradientFunctionJanoschek t =
        d*k*(l-b)*t**(d-1.)*Math.Exp(-k*t**d)
    //Chart to estimate point of maximal slope (inflection point)
    let slopeChart() =
        [time.[0] .. 0.1 .. 8.] |> List.map (fun x -> x, gradientFunctionJanoschek x) |> Chart.Line
    let inflectionPointXValue =
        3.795
    let maximalSlope =
        gradientFunctionJanoschek inflectionPointXValue
    log(2.) / maximalSlope
    
let fittedValuesJanoschek =
    // The parameter were determined locally for saving time during build processes
    let f =  Table.GrowthModels.janoschek.GetFunctionValue (vector [|16.40633962; 23.35535182; 0.01368422994; 2.900857027|])
    [time.[0] .. 0.1 .. Seq.last time]
    |> Seq.map (fun x -> 
        x,f x
        ) 
    |> Chart.Line

let fittedChartJanoschek = 
    fittedValuesJanoschek
    |> Chart.withTemplate ChartTemplates.lightMirrored
    |> Chart.withXAxisStyle "time (h)"
    |> Chart.withYAxisStyle "ln(cells/ml)"
    |> Chart.withTraceInfo "janoschek"

let fittedChartJanoschekS = 
    [
        Chart.Point(time,cellCountLn)
        |> Chart.withTraceInfo "log count"
        fittedChartJanoschek
    ]
    |> Chart.combine
    |> Chart.withTemplate ChartTemplates.lightMirrored
    |> Chart.withXAxisStyle "time (h)"
    |> Chart.withYAxisStyle "ln(cells/ml)"
    |> Chart.withTitle "janoschek"

let generationJanoschek = 
    sprintf "The generation time (Janoschek) is: %.1f min" (generationtimeJanoschek (vector [|16.40633962; 23.35535182; 0.01368422994; 2.900857027|]) * 60.)


In [ ]:
fittedChartJanoschekS


<!-- Plotly chart will be drawn inside this DIV -->

"The generation time (Janoschek) is: 23.0 min"

### Exponential

The exponential model of course can not be applied to the lag phase.

Parameters:

* b: lower asymptote
  

* l: upper asymptote
  

* k: growth rate
  



In [ ]:
let exponentialParams =
    LevenbergMarquardt.estimatedParams
        Table.GrowthModels.exponential
        (createSolverOption 0.001 0.001 10000 [|15.;25.;0.5|])
        0.1
        10.
        time.[6..]
        cellCountLn.[6..]

let fittingFunctionExponential = 
    Table.GrowthModels.exponential.GetFunctionValue exponentialParams


Here is a pre-evaluated version (to save time during the build process, as the solver takes quite some time.)



In [23]:
let generationtimeExponential (expParameters:vector) =
    let b = expParameters.[0]
    let l = expParameters.[1]
    let k = expParameters.[2]
    let gradientFunctionExponential t =
        k*(l-b)*Math.Exp(-k*t)
    let maximalSlope =
        gradientFunctionExponential time.[6]
    log(2.) / maximalSlope

let fittedValuesExp =
    // The parameter were determined locally for saving time during build processes
    let f =  Table.GrowthModels.exponential.GetFunctionValue (vector [|4.813988967; 24.39950361; 0.3939132175|])
    [3.0 .. 0.1 .. Seq.last time]
    |> Seq.map (fun x -> 
        x,f x
        ) 
    |> Chart.Line

let fittedChartExp = 
    fittedValuesExp
    |> Chart.withTemplate ChartTemplates.lightMirrored
    |> Chart.withXAxisStyle "time (h)"
    |> Chart.withYAxisStyle "ln(cells/ml)"
    |> Chart.withTraceInfo "exponential"

let fittedChartExpS = 
    [
        Chart.Point(time,cellCountLn)
        |> Chart.withTraceInfo "log count"
        fittedChartExp
    ]
    |> Chart.combine
    |> Chart.withTemplate ChartTemplates.lightMirrored
    |> Chart.withXAxisStyle "time (h)"
    |> Chart.withYAxisStyle "ln(cells/ml)"
    |> Chart.withTitle "exponential"

let generationExponential = 
    sprintf "The generation time (Exp) is: %.1f min" (generationtimeExponential (vector [|4.813988967; 24.39950361; 0.3939132175|]) * 60.)
    


In [ ]:
fittedChartExpS


<!-- Plotly chart will be drawn inside this DIV -->

"The generation time (Exp) is: 17.6 min"

## Verhulst

The verhulst growth model is a logistic function with a lower asymptote fixed at y=0. A 4 parameter version allows
the lower asymptote to vary from 0.

Note: symmetric with inflection point at 50 % of y axis range

Parameters:

* l: upper asymptote
  

* k: x value at inflection point
  

* d: steepness
  

* b: lower asymptote
  

To apply the 3 parameter verhulst model with a fixed lower asymptote = 0 use the 'verhulst' model instead of 'verhulst4Param'.



In [ ]:
let verhulstParams =
    LevenbergMarquardt.estimatedParams
        Table.GrowthModels.verhulst4Param
        (createSolverOption 0.001 0.001 10000 [|25.;3.5;1.;15.|])
        0.1
        10.
        time
        cellCountLn

let fittingFunctionVerhulst() = 
    Table.GrowthModels.verhulst.GetFunctionValue verhulstParams


Here is a pre-evaluated version (to save time during the build process, as the solver takes quite some time.)



In [26]:
let generationtimeVerhulst (verhulstParameters:vector) =
    let lmax = verhulstParameters.[0]
    let k    = verhulstParameters.[1]
    let d    = verhulstParameters.[2]
    let lmin = verhulstParameters.[3]
    let gradientFunctionVerhulst t =
        ((lmax-lmin)*Math.Exp((k-t)/d))/(d*(Math.Exp((k-t)/d)+1.)**2.)
    let maximalSlope =
        gradientFunctionVerhulst k
    log(2.) / maximalSlope

let fittedValuesVerhulst =
    // The parameter were determined locally for saving time during build processes
    let f =  Table.GrowthModels.verhulst4Param.GetFunctionValue (vector [|23.39504328; 3.577488116; 1.072136278; 15.77380824|])
    [time.[0] .. 0.1 .. Seq.last time]
    |> Seq.map (fun x -> 
        x,f x
        ) 
    |> Chart.Line

let fittedChartVerhulst = 
    fittedValuesVerhulst
    |> Chart.withTemplate ChartTemplates.lightMirrored
    |> Chart.withXAxisStyle "time (h)"
    |> Chart.withYAxisStyle "ln(cells/ml)"
    |> Chart.withTraceInfo "verhulst"

let fittedChartVerhulstS = 
    [
        Chart.Point(time,cellCountLn)
        |> Chart.withTraceInfo "log count"
        fittedChartVerhulst
    ]
    |> Chart.combine
    |> Chart.withTemplate ChartTemplates.lightMirrored
    |> Chart.withXAxisStyle "time (h)"
    |> Chart.withYAxisStyle "ln(cells/ml)"
    |> Chart.withTitle "verhulst"

let generationVerhulst = 
    sprintf "The generation time (Verhulst) is: %.1f min" (generationtimeVerhulst (vector [|23.39504328; 3.577488116; 1.072136278; 15.77380824|]) * 60.)


In [ ]:
fittedChartVerhulstS


<!-- Plotly chart will be drawn inside this DIV -->

"The generation time (Verhulst) is: 23.4 min"

## Morgan-Mercer-Flodin

Parameters:

* b: count at t0
  

* l: upper asymptote
  

* k: growth rate
  

* d: influences the inflection point position
  



In [ ]:
let morganMercerFlodinParams =
    LevenbergMarquardt.estimatedParams
        Table.GrowthModels.morganMercerFlodin
        (createSolverOption 0.001 0.001 10000 [|15.;25.;0.2;3.|])
        0.1
        10.
        time
        cellCountLn

let fittingFunctionMMF() = 
    Table.GrowthModels.morganMercerFlodin.GetFunctionValue morganMercerFlodinParams


Here is a pre-evaluated version (to save time during the build process, as the solver takes quite some time.)



In [29]:
let generationtimeMmf (mmfParameters:vector) =
    let b = mmfParameters.[0]
    let l = mmfParameters.[1]
    let k = mmfParameters.[2]
    let d = mmfParameters.[3]
    let gradientFunctionMmf t =
        (d*(l-b)*(k*t)**d)/(t*((k*t)**d+1.)**2.)
    //Chart to estimate point of maximal slope (inflection point)
    let slopeChart() =
        [time.[0] .. 0.1 .. 8.] |> List.map (fun x -> x, gradientFunctionMmf x) |> Chart.Line
    let inflectionPointXValue =
        3.45
    let maximalSlope =
        gradientFunctionMmf inflectionPointXValue
    log(2.) / maximalSlope

let generationMmf = 
    sprintf "The generation time (MMF) is: %.1f min" (generationtimeMmf (vector [|16.46099291; 24.00147463; 0.2500698772; 3.741048641|]) * 60.)

let fittedValuesMMF =
    // The parameter were determined locally for saving time during build processes
    let f =  Table.GrowthModels.morganMercerFlodin.GetFunctionValue (vector [|16.46099291; 24.00147463; 0.2500698772; 3.741048641|])
    [time.[0] .. 0.1 .. Seq.last time]
    |> Seq.map (fun x -> 
        x,f x
        ) 
    |> Chart.Line

let fittedChartMMF = 
    [
        Chart.Point(time,cellCountLn)
        |> Chart.withTraceInfo "log count"
        fittedValuesMMF |> Chart.withTraceInfo "morganMercerFlodin"
    ]
    |> Chart.combine
    |> Chart.withTemplate ChartTemplates.lightMirrored
    |> Chart.withXAxisStyle "time (h)"
    |> Chart.withYAxisStyle "ln(cells/ml)"
    |> Chart.withTitle "morganMercerFlodin"


In [ ]:
fittedChartMMF


<!-- Plotly chart will be drawn inside this DIV -->

"The generation time (MMF) is: 21.9 min"

## von Bertalanffy

Since this model expects a x axis crossing of the data it cannot be applied to the given data.

Parameters:

* l: upper asymptote
  

* k: growth rate
  

* t0: x axis crossing
  

## Comparison between all models

### Fit function



In [32]:
let combinedChart =
    [
        
        Chart.Point(time,cellCountLn) |> Chart.withTraceInfo "log count"
        Chart.Line(fittedValues)|> Chart.withTraceInfo "regression line"
        fittedValuesGompertz    |> Chart.withTraceInfo "Gompertz"
        fittedValuesRichards    |> Chart.withTraceInfo "Richards"
        fittedValuesWeibull     |> Chart.withTraceInfo "Weibull"
        fittedValuesJanoschek   |> Chart.withTraceInfo "Janoschek"
        fittedValuesExp         |> Chart.withTraceInfo "Exponential"
        fittedValuesVerhulst    |> Chart.withTraceInfo "Verhulst"
        fittedValuesMMF         |> Chart.withTraceInfo "MorganMercerFlodin"
    ]
    |> Chart.combine
    |> Chart.withTemplate ChartTemplates.lightMirrored
    |> Chart.withXAxisStyle "time (h)"
    |> Chart.withYAxisStyle "ln(cells/ml)"


In [ ]:
combinedChart


<!-- Plotly chart will be drawn inside this DIV -->

### Generation time



In [34]:
let generationTimeTable =
    let header = ["<b>Model</b>";"<b>Generation time (min)"]
    let rows = 
        [
            ["manual selection (regression line)";      sprintf "%.1f" ((log(2.)/1.5150)* 60.)]    
            ["Gompertz";    sprintf "%.1f" (60. * (generationtime gompertzParams log))]    
            ["Richards";    sprintf "%.1f" (generationtimeRichards (vector [|23.25211263; 7.053516315; 5.646889803; 111.0132522|]) * 60.)]       
            ["Weibull";     sprintf "%.1f" (generationtimeWeibull (vector [|16.40632433; 23.35537293; 0.2277752116; 2.900806071|]) * 60.)  ]       
            ["Janoschek";   sprintf "%.1f" (generationtimeJanoschek (vector [|16.40633962; 23.35535182; 0.01368422994; 2.900857027|]) * 60.)]    
            ["Exponential"; sprintf "%.1f" (generationtimeExponential (vector [|4.813988967; 24.39950361; 0.3939132175|]) * 60.)]
            ["Verhulst";    sprintf "%.1f" (generationtimeVerhulst (vector [|23.39504328; 3.577488116; 1.072136278; 15.77380824|]) * 60.)] 
            ["MMF";         sprintf "%.1f" (generationtimeMmf (vector [|16.46099291; 24.00147463; 0.2500698772; 3.741048641|]) * 60.)] 
        ]
    
    Chart.Table(
        header, 
        rows,
        HeaderFillColor = Color.fromHex "#45546a",
        CellsFillColor = Color.fromColors [Color.fromHex "#deebf7";Color.fromString "lightgrey"]
        )


<!-- Plotly chart will be drawn inside this DIV -->

## Model examples

<div><div id="6cfc22c7-ab83-4a69-9a40-c2aef4243cbf"><!-- Plotly chart will be drawn inside this DIV --></div><script type="text/javascript">var renderPlotly_6cfc22c7ab834a699a40c2aef4243cbf = function() {
    var data = [{"type":"scatter","mode":"lines","x":[0.0,0.1,0.2,0.30000000000000004,0.4,0.5,0.6000000000000001,0.7000000000000001,0.8,0.9,1.0,1.1,1.2000000000000002,1.3,1.4000000000000001,1.5,1.6,1.7000000000000002,1.8,1.9000000000000001,2.0,2.1,2.2,2.3000000000000003,2.4000000000000004,2.5,2.6,2.7,2.8000000000000003,2.9000000000000004,3.0,3.1,3.2,3.3000000000000003,3.4000000000000004,3.5,3.6,3.7,3.8000000000000003,3.9000000000000004,4.0,4.1000000000000005,4.2,4.3,4.4,4.5,4.6000000000000005,4.7,4.800000000000001,4.9,5.0,5.1000000000000005,5.2,5.300000000000001,5.4,5.5,5.6000000000000005,5.7,5.800000000000001,5.9,6.0,6.1000000000000005,6.2,6.300000000000001,6.4,6.5,6.6000000000000005,6.7,6.800000000000001,6.9,7.0,7.1000000000000005,7.2,7.300000000000001,7.4,7.5,7.6000000000000005,7.7,7.800000000000001,7.9,8.0,8.1,8.200000000000001,8.3,8.4,8.5,8.6,8.700000000000001,8.8,8.9,9.0,9.1,9.200000000000001,9.3,9.4,9.5,9.600000000000001,9.700000000000001,9.8,9.9,10.0],"y":[5.173320140087645,5.227988909313028,5.294393964201264,5.3736274450058765,5.466606443368613,5.574034359286964,5.696372090346445,5.83382023919366,5.986312651583164,6.153520809993811,6.3348679665808385,6.529551436605267,6.736571197084235,6.9547628330075035,7.182832919154284,7.419395085858047,7.663005256300666,7.912194827187897,8.165500864608246,8.421492679124922,8.678794411714424,8.936103494143511,9.19220503815091,9.445982356003332,9.69642392241606,9.942627158384465,10.183799456323944,10.419256878749074,10.648420955220312,10.87081397975489,11.086053178044065,11.293844074631004,11.493973347927412,11.686301418109352,11.870754971410982,12.04731958544079,12.216032584720264,12.37697622415304,12.530271270745214,12.676071030568988,12.814555848495635,12.94592809232228,13.070407620225863,13.188227720615966,13.299631506042523,13.404868737475784,13.504193051669713,13.597859562133618,13.68612280319187,13.7692349864653,13.847444539651772,13.92099489854155,13.990123524628682,14.055061122352948,14.116031031830829,14.173248774832096,14.22692173367091,14.277248944562173,14.32442098881127,14.3686199669353,14.410019542440478,14.44878504349402,14.485073612125435,14.519034391870399,14.550808745932631,14.580530498989612,14.608326196711607,14.634315377907322,14.658610854960113,14.681318998883455,14.702540025910624,14.722368283048228,14.740892530473195,14.75819621904473,14.774357761542436,14.789450796535009,14.803544444036001,14.8167035523185,14.82898893544399,14.840457601215642,14.851162969396578,14.861155080142401,14.870480792687232,14.879183974396158,14.887305680356693,14.89488432372945,14.901955837115379,14.90855382522544,14.91470970915942,14.92045286261519,14.925810740358976,14.930808999291914,14.935471612449403,14.939820976267683,14.943878011447785,14.947662257740502,14.951191962968197,14.95448416658992,14.957554778106394,14.960418650590503,14.963089649617828],"marker":{},"line":{},"name":"vector [|5.0; 0.7; 10.0; 2.0|]"},{"type":"scatter","mode":"lines","x":[0.0,0.1,0.2,0.30000000000000004,0.4,0.5,0.6000000000000001,0.7000000000000001,0.8,0.9,1.0,1.1,1.2000000000000002,1.3,1.4000000000000001,1.5,1.6,1.7000000000000002,1.8,1.9000000000000001,2.0,2.1,2.2,2.3000000000000003,2.4000000000000004,2.5,2.6,2.7,2.8000000000000003,2.9000000000000004,3.0,3.1,3.2,3.3000000000000003,3.4000000000000004,3.5,3.6,3.7,3.8000000000000003,3.9000000000000004,4.0,4.1000000000000005,4.2,4.3,4.4,4.5,4.6000000000000005,4.7,4.800000000000001,4.9,5.0,5.1000000000000005,5.2,5.300000000000001,5.4,5.5,5.6000000000000005,5.7,5.800000000000001,5.9,6.0,6.1000000000000005,6.2,6.300000000000001,6.4,6.5,6.6000000000000005,6.7,6.800000000000001,6.9,7.0,7.1000000000000005,7.2,7.300000000000001,7.4,7.5,7.6000000000000005,7.7,7.800000000000001,7.9,8.0,8.1,8.200000000000001,8.3,8.4,8.5,8.6,8.700000000000001,8.8,8.9,9.0,9.1,9.200000000000001,9.3,9.4,9.5,9.600000000000001,9.700000000000001,9.8,9.9,10.0],"y":[7.0034092489446556,7.005921415766141,7.0099079242237705,7.0160112763795786,7.025048237828445,7.038017981788639,7.0560983251662055,7.080628380340149,7.113077300242323,7.15500025897238,7.207984168105174,7.273586691175634,7.353272757041517,7.448352934007051,7.559927732042335,7.688841231144357,7.835646508415735,8.00058428703239,8.183575181899798,8.384224971992573,8.601841559897007,8.835461723926322,9.083885436501081,9.345715399609004,9.619399502985141,9.903274103029656,10.1956063075608,10.494633792625477,10.798601037529897,11.105791214949907,11.414553294057308,11.723324192972214,12.030646045781092,12.335178827204,12.63570870689927,12.931152590061359,13.220559347588733,13.50310825449889,13.778105146264373,14.04497677570587,14.303263813652876,14.552612889557206,14.792768017512895,15.023561701731222,15.244905965693178,15.45678350252895,15.659239101664317,15.85237146898365,16.03632552489426,16.211285236682784,16.37746701819476,16.535113710786735,16.684489144271037,16.82587326473916,16.959557807251027,17.08584248497094,17.205031662003655,17.31743147456034,17.423347363830246,17.52308198375836,17.616933447582127,17.70519387824986,17.78814822955442,17.866073346823537,17.939237238196995,18.007898529798517,18.07230608040509,18.13269873347461,18.18930518657352,18.242343960322362,18.292023450928575,18.33854205219282,18.382088334550524,18.422841270244476,18.460970495119156,18.496636598787536,18.529991436053926,18.561178453488786,18.590333025952134,18.617582798660145,18.64304803109275,18.666841939657875,18.68907103656783,18.709835462853675,18.729229313850922,18.747340955842013,18.764253332843204,18.780044262782198,18.794786722532788,18.80854912145877,18.821395563275892,18.833386096170884,18.84457695122468,18.855020769275388,18.86476681642803,18.87386118847534,18.882347004538456,18.890264590270526,18.897651650991303,18.90454343513823,18.91097288843077],"marker":{},"line":{},"name":"vector [|7.0; 0.7; 12.0; 3.0|]"},{"type":"scatter","mode":"lines","x":[0.0,0.1,0.2,0.30000000000000004,0.4,0.5,0.6000000000000001,0.7000000000000001,0.8,0.9,1.0,1.1,1.2000000000000002,1.3,1.4000000000000001,1.5,1.6,1.7000000000000002,1.8,1.9000000000000001,2.0,2.1,2.2,2.3000000000000003,2.4000000000000004,2.5,2.6,2.7,2.8000000000000003,2.9000000000000004,3.0,3.1,3.2,3.3000000000000003,3.4000000000000004,3.5,3.6,3.7,3.8000000000000003,3.9000000000000004,4.0,4.1000000000000005,4.2,4.3,4.4,4.5,4.6000000000000005,4.7,4.800000000000001,4.9,5.0,5.1000000000000005,5.2,5.300000000000001,5.4,5.5,5.6000000000000005,5.7,5.800000000000001,5.9,6.0,6.1000000000000005,6.2,6.300000000000001,6.4,6.5,6.6000000000000005,6.7,6.800000000000001,6.9,7.0,7.1000000000000005,7.2,7.300000000000001,7.4,7.5,7.6000000000000005,7.7,7.800000000000001,7.9,8.0,8.1,8.200000000000001,8.3,8.4,8.5,8.6,8.700000000000001,8.8,8.9,9.0,9.1,9.200000000000001,9.3,9.4,9.5,9.600000000000001,9.700000000000001,9.8,9.9,10.0],"y":[5.0001631906697614,5.000380855997698,5.000832775721082,5.00171463925836,5.003339668189792,5.006179789893311,5.010906750426033,5.018426726003544,5.029901330495193,5.046748604305171,5.070619615622094,5.103349368606419,5.146884204956612,5.203191131423192,5.274156920882616,5.361486049131355,5.466606443368613,5.590590779810693,5.734099001210889,5.897345235159247,6.080089776965916,6.281654578121422,6.500958964778625,6.736571197084235,6.986770966209869,7.2496179354991845,7.5230218467494865,7.804810398781081,8.092791935628245,8.384810847947222,8.678794411714424,8.972790508339187,9.26499625995572,9.553778062966366,9.83768381671032,10.115448336890417,10.385993034821212,10.648420955220312,10.902008218129055,11.146192823484993,11.380561665820187,11.604836484272711,11.818859349213543,12.02257816815645,12.216032584720264,12.399340547836061,12.572685745265066,12.736306025885975,12.890482878468973,13.03553198966916,13.171794869390288,13.299631506042523,13.41941399611202,13.531521080525966,13.636333513311161,13.734230184931167,13.825584922522635,13.910763891260704,13.990123524628684,14.064008915934691,14.132752608601853,14.196673728244944,14.25607740509879,14.311254440809103,14.362481178812722,14.410019542440478,14.45411720942252,14.49500789563767,14.532911724718193,14.56803566350509,14.600574006362038,14.630708894019582,14.658610854960113,14.684439359395654,14.708343377661734,14.730461936379186,14.750924667047212,14.769852342850555,14.787357400413603,14.803544444036001,14.818510730616666,14.832346634033298,14.84513608820795,14.856957008469196,14.867881691130581,14.877977191453125,14.887305680356693,14.89592478039831,14.90388788165257,14.911244438215755,14.91804024611649,14.924317703456044,14.930116053624648,14.935471612449403,14.94041798012759,14.94498623878831,14.949205136507743,14.953101258580183,14.956699186820009,14.96002164763979,14.963089649617828],"marker":{},"line":{},"name":"vector [|5.0; 0.8; 10.0; 3.0|]"}];
    var layout = {"width":600,"height":600,"template":{"layout":{"paper_bgcolor":"white","plot_bgcolor":"white","xaxis":{"ticks":"inside","mirror":"all","showline":true,"zeroline":true},"yaxis":{"ticks":"inside","mirror":"all","showline":true,"zeroline":true}},"data":{}},"title":{"text":"Gompertz"}};
    var config = {"responsive":true};
    Plotly.newPlot('6cfc22c7-ab83-4a69-9a40-c2aef4243cbf', data, layout, config);
};
renderPlotly_6cfc22c7ab834a699a40c2aef4243cbf();
</script></div>

<div><div id="e0256464-faf7-4567-bfce-9c5bde4b50c8"><!-- Plotly chart will be drawn inside this DIV --></div><script type="text/javascript">var renderPlotly_e0256464faf74567bfce9c5bde4b50c8 = function() {
    var data = [{"type":"scatter","mode":"lines","x":[0.0,0.1,0.2,0.30000000000000004,0.4,0.5,0.6000000000000001,0.7000000000000001,0.8,0.9,1.0,1.1,1.2000000000000002,1.3,1.4000000000000001,1.5,1.6,1.7000000000000002,1.8,1.9000000000000001,2.0,2.1,2.2,2.3000000000000003,2.4000000000000004,2.5,2.6,2.7,2.8000000000000003,2.9000000000000004,3.0,3.1,3.2,3.3000000000000003,3.4000000000000004,3.5,3.6,3.7,3.8000000000000003,3.9000000000000004,4.0,4.1000000000000005,4.2,4.3,4.4,4.5,4.6000000000000005,4.7,4.800000000000001,4.9,5.0,5.1000000000000005,5.2,5.300000000000001,5.4,5.5,5.6000000000000005,5.7,5.800000000000001,5.9,6.0,6.1000000000000005,6.2,6.300000000000001,6.4,6.5,6.6000000000000005,6.7,6.800000000000001,6.9,7.0,7.1000000000000005,7.2,7.300000000000001,7.4,7.5,7.6000000000000005,7.7,7.800000000000001,7.9,8.0,8.1,8.200000000000001,8.3,8.4,8.5,8.6,8.700000000000001,8.8,8.9,9.0,9.1,9.200000000000001,9.3,9.4,9.5,9.600000000000001,9.700000000000001,9.8,9.9,10.0],"y":[0.0022409815509030775,0.0026695607940113828,0.0031801041958827445,0.003788287091778407,0.004512782665522462,0.005375835276697457,0.0064039434344970786,0.007628673387744187,0.009087628310921587,0.010825602843364595,0.012895958429705765,0.015362261688976564,0.018300236115600707,0.02180008703581828,0.025969271203229397,0.03093579606900496,0.03685215002504015,0.043899984291231875,0.05229569019596421,0.0622970430903364,0.07421111688572972,0.08840371221645585,0.1053105877021524,0.12545083914497224,0.1494428374436636,0.1780232145639332,0.21206948048106186,0.2526269654726484,0.3009409148919874,0.3584947216313678,0.42705546964592916,0.5087281856968474,0.6060204621996583,0.7219194285475795,0.8599834179604808,1.0244511054239305,1.2203713742827054,1.453757674959153,1.7317710781938958,2.062936355549933,2.4573946347525637,2.927192991818566,3.4866021764361954,4.152429997767805,4.944239750087805,5.8842449707047875,6.996336496566623,8.303025546199063,9.817856176793093,11.529427750744581,13.37480609952844,15.214060489731999,16.84666666475018,18.102815670118726,18.94049086746596,19.43788368405531,19.710849483907577,19.85378044182701,19.926718940699708,19.9634415819227,19.981803823722494,19.99095369505761,19.995505180077767,19.997767307084054,19.99889112172978,19.999449308922674,19.999726525427484,19.999864194209284,19.999932560263836,19.999966510276007,19.99998336946019,19.999991741509692,19.999995898952953,19.999997963479778,19.99999898869386,19.9999994978002,19.99999975061495,19.99999987615905,19.999999938502402,19.999999969461197,19.99999998483488,19.999999992469224,19.99999999626033,19.999999998142933,19.999999999077808,19.99999999954205,19.99999999977259,19.999999999887073,19.999999999943924,19.999999999972154,19.999999999986173,19.999999999993136,19.99999999999659,19.99999999999831,19.99999999999916,19.99999999999958,19.999999999999794,19.999999999999897,19.99999999999995,19.999999999999975,19.99999999999999],"marker":{},"line":{},"name":"vector [|20.0; 7.0; 5.0; 5.0|]"},{"type":"scatter","mode":"lines","x":[0.0,0.1,0.2,0.30000000000000004,0.4,0.5,0.6000000000000001,0.7000000000000001,0.8,0.9,1.0,1.1,1.2000000000000002,1.3,1.4000000000000001,1.5,1.6,1.7000000000000002,1.8,1.9000000000000001,2.0,2.1,2.2,2.3000000000000003,2.4000000000000004,2.5,2.6,2.7,2.8000000000000003,2.9000000000000004,3.0,3.1,3.2,3.3000000000000003,3.4000000000000004,3.5,3.6,3.7,3.8000000000000003,3.9000000000000004,4.0,4.1000000000000005,4.2,4.3,4.4,4.5,4.6000000000000005,4.7,4.800000000000001,4.9,5.0,5.1000000000000005,5.2,5.300000000000001,5.4,5.5,5.6000000000000005,5.7,5.800000000000001,5.9,6.0,6.1000000000000005,6.2,6.300000000000001,6.4,6.5,6.6000000000000005,6.7,6.800000000000001,6.9,7.0,7.1000000000000005,7.2,7.300000000000001,7.4,7.5,7.6000000000000005,7.7,7.800000000000001,7.9,8.0,8.1,8.200000000000001,8.3,8.4,8.5,8.6,8.700000000000001,8.8,8.9,9.0,9.1,9.200000000000001,9.3,9.4,9.5,9.600000000000001,9.700000000000001,9.8,9.9,10.0],"y":[0.974158197231847,1.0298096580792857,1.0886403613774658,1.1508319300775813,1.2165763628403772,1.286076626777002,1.3595472840517822,1.4372151542813545,1.5193200147750874,1.6061153407785236,1.6978690880050196,1.7948645198712372,1.897401081990005,2.005795326619722,2.1203818899232587,2.2415145250516124,2.3695671942385954,2.504935223272742,2.6480365219012283,2.799312873917212,2.959231300885083,3.1282855036644293,3.3069973860970565,3.495918665411242,3.6956325740538065,3.9067556577489198,4.129939674541604,4.365873599306481,4.615285737501129,4.878945950491046,5.157667991998601,5.452311950135594,5.763786780331548,6.093052898241695,6.4411247731375205,6.809073412167566,7.198028538248936,7.609180111432937,8.043778577171443,8.503132761556621,8.988603529579644,9.501589932070484,10.04350217584828,10.615711676773154,11.219461617548482,11.855710260486694,12.524861873593945,13.226315490886885,13.957733745370632,14.713922382467723,15.485273653622542,16.25598555556332,17.002871719599682,17.69641376550934,18.30583074408495,18.807737895342626,19.193935705646627,19.472817695198422,19.66378173994186,19.78935517941602,19.869596019375884,19.919892712888572,19.951029896399767,19.97015511924603,19.981845036376836,19.98896879608872,19.993301977106242,19.99593476723023,19.9975333257269,19.99850352341642,19.999092207460865,19.99944934682539,19.999665993875926,19.99979740838925,19.999877119527923,19.99992546832532,19.99995479392276,19.999972581006226,19.99998336949476,19.999989913072184,19.999993881962947,19.999996289220718,19.999997749297773,19.99999863487979,19.99999917201263,19.999999497800232,19.999999695400426,19.999999815251016,19.999999887944075,19.999999932034644,19.99999995877693,19.999999974996943,19.99999998483488,19.99999999080189,19.999999994421064,19.999999996616204,19.999999997947626,19.99999999875517,19.999999999244974,19.99999999954205,19.99999999972224],"marker":{},"line":{},"name":"vector [|20.0; 5.0; 5.0; 10.0|]"},{"type":"scatter","mode":"lines","x":[0.0,0.1,0.2,0.30000000000000004,0.4,0.5,0.6000000000000001,0.7000000000000001,0.8,0.9,1.0,1.1,1.2000000000000002,1.3,1.4000000000000001,1.5,1.6,1.7000000000000002,1.8,1.9000000000000001,2.0,2.1,2.2,2.3000000000000003,2.4000000000000004,2.5,2.6,2.7,2.8000000000000003,2.9000000000000004,3.0,3.1,3.2,3.3000000000000003,3.4000000000000004,3.5,3.6,3.7,3.8000000000000003,3.9000000000000004,4.0,4.1000000000000005,4.2,4.3,4.4,4.5,4.6000000000000005,4.7,4.800000000000001,4.9,5.0,5.1000000000000005,5.2,5.300000000000001,5.4,5.5,5.6000000000000005,5.7,5.800000000000001,5.9,6.0,6.1000000000000005,6.2,6.300000000000001,6.4,6.5,6.6000000000000005,6.7,6.800000000000001,6.9,7.0,7.1000000000000005,7.2,7.300000000000001,7.4,7.5,7.6000000000000005,7.7,7.800000000000001,7.9,8.0,8.1,8.200000000000001,8.3,8.4,8.5,8.6,8.700000000000001,8.8,8.9,9.0,9.1,9.200000000000001,9.3,9.4,9.5,9.600000000000001,9.700000000000001,9.8,9.9,10.0],"y":[1.0197383819668984,1.072021486827054,1.1269852037953325,1.1847669707934763,1.245511272336159,1.3093700008175297,1.3765028363213194,1.4470776459042207,1.5212709033509648,1.5992681304507126,1.6812643608981515,1.7674646279793285,1.8580844772616572,1.9533505055701048,2.0535009275972858,2.1587861715642536,2.2694695054214833,2.3858276951558275,2.5081516968495436,2.6367473842218563,2.7719363134721253,2.9140565273369265,3.0634634003710066,3.220530527564631,3.3856506585168953,3.559236679495715,3.7417226458292667,3.933564867187169,4.135243048416175,4.347261488679213,4.570150341676438,4.804466939634213,5.050797183387549,5.309756999946582,5.581993866776901,5.868188397259176,6.169055971507181,6.48534837473771,6.817855358625124,7.167405941865132,7.534869055976843,7.921152697440405,8.327199806394022,8.75397710561312,9.202448951654278,9.673519498228295,10.16790837336336,10.685888561623829,11.226745363389561,11.787697542132333,12.361886179183463,12.93512888797469,13.482281787624594,13.96737103440917,14.354401460874774,14.62693093823711,14.79720211691163,14.894123645299858,14.946017499196131,14.972828176411097,14.986414500894368,14.993230560169208,14.996632666399496,14.998326414151178,14.999168571747187,14.999587038540858,14.999794908091893,14.999898149114879,14.999949421050626,14.999974882917295,14.999987527146999,14.999993806145058,14.999996924217868,14.999998472610612,14.999999241520586,14.999999623350197,14.999999812961226,14.999999907119289,14.999999953876802,14.999999977095898,14.99999998862616,14.999999994351917,14.999999997195244,14.999999998607201,14.999999999308356,14.99999999965654,14.999999999829443,14.999999999915305,14.99999999995794,14.999999999979114,14.999999999989628,14.999999999994849,14.999999999997442,14.99999999999873,14.99999999999937,14.999999999999687,14.999999999999845,14.999999999999924,14.999999999999961,14.999999999999982,14.99999999999999],"marker":{},"line":{},"name":"vector [|15.0; 7.0; 5.0; 15.0|]"}];
    var layout = {"width":600,"height":600,"template":{"layout":{"paper_bgcolor":"white","plot_bgcolor":"white","xaxis":{"ticks":"inside","mirror":"all","showline":true,"zeroline":true},"yaxis":{"ticks":"inside","mirror":"all","showline":true,"zeroline":true}},"data":{}},"title":{"text":"Richards"}};
    var config = {"responsive":true};
    Plotly.newPlot('e0256464-faf7-4567-bfce-9c5bde4b50c8', data, layout, config);
};
renderPlotly_e0256464faf74567bfce9c5bde4b50c8();
</script></div>

<div><div id="77d94404-95ab-4531-bf02-71e4a93567ed"><!-- Plotly chart will be drawn inside this DIV --></div><script type="text/javascript">var renderPlotly_77d9440495ab4531bf0271e4a93567ed = function() {
    var data = [{"type":"scatter","mode":"lines","x":[-6.0,-5.9,-5.8,-5.7,-5.6,-5.5,-5.4,-5.3,-5.2,-5.1,-5.0,-4.9,-4.8,-4.7,-4.6,-4.5,-4.4,-4.3,-4.2,-4.1,-4.0,-3.9,-3.8,-3.6999999999999997,-3.5999999999999996,-3.5,-3.4,-3.3,-3.1999999999999997,-3.0999999999999996,-3.0,-2.9,-2.8,-2.6999999999999997,-2.5999999999999996,-2.5,-2.4,-2.3,-2.1999999999999997,-2.0999999999999996,-2.0,-1.8999999999999995,-1.7999999999999998,-1.7000000000000002,-1.5999999999999996,-1.5,-1.3999999999999995,-1.2999999999999998,-1.1999999999999993,-1.0999999999999996,-1.0,-0.8999999999999995,-0.7999999999999998,-0.6999999999999993,-0.5999999999999996,-0.5,-0.39999999999999947,-0.2999999999999998,-0.1999999999999993,-0.09999999999999964,0.0,0.10000000000000053,0.20000000000000018,0.3000000000000007,0.40000000000000036,0.5,0.6000000000000005,0.7000000000000002,0.8000000000000007,0.9000000000000004,1.0,1.1000000000000005,1.2000000000000002,1.3000000000000007,1.4000000000000004,1.5,1.6000000000000005,1.7000000000000002,1.8000000000000007,1.9000000000000004,2.0,2.0999999999999996,2.200000000000001,2.3000000000000007,2.4000000000000004,2.5,2.5999999999999996,2.700000000000001,2.8000000000000007,2.9000000000000004,3.0,3.0999999999999996,3.200000000000001,3.3000000000000007,3.4000000000000004,3.5,3.6000000000000014,3.700000000000001,3.8000000000000007,3.9000000000000004,4.0,4.100000000000001,4.200000000000001,4.300000000000001,4.4,4.5,4.600000000000001,4.700000000000001,4.800000000000001,4.9,5.0,5.100000000000001,5.200000000000001,5.300000000000001,5.4,5.5,5.600000000000001,5.700000000000001,5.800000000000001,5.9,6.0],"y":[9.278090755753732E-16,1.6905783237468957E-15,3.080434458607696E-15,5.612917322419411E-15,1.022740164207282E-14,1.8635539492461192E-14,3.395616362114421E-14,6.187215614552955E-14,1.1273839931150905E-13,2.0542270864922287E-13,3.7430446075105436E-13,6.820269030354462E-13,1.242733324265492E-12,2.264405987709063E-12,4.126012377513638E-12,7.518074038212153E-12,1.369879776573246E-11,2.496077230856505E-11,4.54813335000005E-11,8.287200177326696E-11,1.510016706030423E-10,2.7514061714359006E-10,5.013330725241315E-10,9.13474105244228E-10,1.6644231412969405E-09,3.03269012389458E-09,5.525708764357259E-09,1.0067974146761648E-08,1.8343757052054986E-08,3.342133720782747E-08,6.088985764783258E-08,1.1092940031963897E-07,2.0207944880088527E-07,3.680969703742358E-07,6.70431764402965E-07,1.2209067303745816E-06,2.2229181244456478E-06,4.04619864606906E-06,7.362288603290728E-06,1.3389531296614122E-05,2.4334969898007E-05,4.418854711250906E-05,8.0143574326316E-05,0.0001451206428898351,0.00026221215009896446,0.00047241383951789886,0.0008478506432792254,0.0015138831941066914,0.002684881435418271,0.004719509270093542,0.008200570518363271,0.014038765230316818,0.023584471372180657,0.03870305644084781,0.061731420376382844,0.09521022764755324,0.14132455841667932,0.20112570146581651,0.27381146423732217,0.35645686765475887,0.4444444444444444,0.5324753721790988,0.6157217698211368,0.6906581119263558,0.7553590647074769,0.8093385186735835,0.8531478202604504,0.8879375545991559,0.9151011352412677,0.9360362808926337,0.9520121551919472,0.9641125870704268,0.973226188530969,0.9800612947124725,0.9851714033257601,0.9889828800631293,0.9918207722346132,0.9939310182946776,0.9954986727466336,0.9966624124653979,0.9975258483793887,0.9981662211419222,0.9986410186556701,0.9989929757831075,0.9992538320264273,0.9994471449720269,0.9995903908987,0.9996965299499467,0.9997751705941217,0.9998334349998252,0.9998766016174585,0.9999085820369322,0.9999322747037401,0.9999498272060046,0.9999628307175145,0.999972464119332,0.9999796008086891,0.9999848878474651,0.9999888046091607,0.9999917062324295,0.9999938558159603,0.9999954482710756,0.9999966279932937,0.9999975019543544,0.9999981494013709,0.9999986290423234,0.9999989843693028,0.9999992476021254,0.9999994426098635,0.9999995870751861,0.9999996940977499,0.9999997733820257,0.9999998321172681,0.9999998756294094,0.9999999078639981,0.9999999317439698,0.9999999494346886,0.9999999625402954,0.9999999722491683,0.9999999794416783,0.9999999847700203],"marker":{},"line":{},"name":"vector [|0.0; 1.0; 3.0; 0.5; 0.5; 1.0; 0.0|]"},{"type":"scatter","mode":"lines","x":[-6.0,-5.9,-5.8,-5.7,-5.6,-5.5,-5.4,-5.3,-5.2,-5.1,-5.0,-4.9,-4.8,-4.7,-4.6,-4.5,-4.4,-4.3,-4.2,-4.1,-4.0,-3.9,-3.8,-3.6999999999999997,-3.5999999999999996,-3.5,-3.4,-3.3,-3.1999999999999997,-3.0999999999999996,-3.0,-2.9,-2.8,-2.6999999999999997,-2.5999999999999996,-2.5,-2.4,-2.3,-2.1999999999999997,-2.0999999999999996,-2.0,-1.8999999999999995,-1.7999999999999998,-1.7000000000000002,-1.5999999999999996,-1.5,-1.3999999999999995,-1.2999999999999998,-1.1999999999999993,-1.0999999999999996,-1.0,-0.8999999999999995,-0.7999999999999998,-0.6999999999999993,-0.5999999999999996,-0.5,-0.39999999999999947,-0.2999999999999998,-0.1999999999999993,-0.09999999999999964,0.0,0.10000000000000053,0.20000000000000018,0.3000000000000007,0.40000000000000036,0.5,0.6000000000000005,0.7000000000000002,0.8000000000000007,0.9000000000000004,1.0,1.1000000000000005,1.2000000000000002,1.3000000000000007,1.4000000000000004,1.5,1.6000000000000005,1.7000000000000002,1.8000000000000007,1.9000000000000004,2.0,2.0999999999999996,2.200000000000001,2.3000000000000007,2.4000000000000004,2.5,2.5999999999999996,2.700000000000001,2.8000000000000007,2.9000000000000004,3.0,3.0999999999999996,3.200000000000001,3.3000000000000007,3.4000000000000004,3.5,3.6000000000000014,3.700000000000001,3.8000000000000007,3.9000000000000004,4.0,4.100000000000001,4.200000000000001,4.300000000000001,4.4,4.5,4.600000000000001,4.700000000000001,4.800000000000001,4.9,5.0,5.100000000000001,5.200000000000001,5.300000000000001,5.4,5.5,5.600000000000001,5.700000000000001,5.800000000000001,5.9,6.0],"y":[1.4130514288696996E-23,2.5747475744553637E-23,4.69149596165833E-23,8.548462993652897E-23,1.5576315135076726E-22,2.838189664818252E-22,5.171518747278382E-22,9.423121535837985E-22,1.7170046908446423E-21,3.128586527456011E-21,5.700656330102911E-21,1.0387273073096853E-20,1.892684554993029E-20,3.448696110529539E-20,6.283934019013002E-20,1.1450074314448632E-19,2.0863395669285372E-19,3.8015585476841117E-19,6.926891297531995E-19,1.2621618854144229E-18,2.299808898742048E-18,4.190525027256562E-18,7.635634424734096E-18,1.3913033011316062E-17,2.535119895420722E-17,4.619289606415982E-17,8.416894396271135E-17,1.5336581421894663E-16,2.794507310039972E-16,5.091924247892182E-16,9.278090755753732E-16,1.6905783237468957E-15,3.080434458607696E-15,5.612917322419452E-15,1.022740164207282E-14,1.8635539492461192E-14,3.395616362114421E-14,6.187215614552955E-14,1.1273839931150987E-13,2.0542270864922287E-13,3.7430446075105436E-13,6.820269030354488E-13,1.242733324265492E-12,2.264405987709063E-12,4.126012377513638E-12,7.518074038212153E-12,1.369879776573251E-11,2.496077230856505E-11,4.5481333500000826E-11,8.287200177326696E-11,1.510016706030423E-10,2.7514061714359006E-10,5.013330725241315E-10,9.134741052442312E-10,1.6644231412969405E-09,3.03269012389458E-09,5.525708764357259E-09,1.0067974146761648E-08,1.834375705205505E-08,3.342133720782747E-08,6.088985764783258E-08,1.1092940031963897E-07,2.0207944880088527E-07,3.6809697037423714E-07,6.70431764402965E-07,1.2209067303745816E-06,2.2229181244456516E-06,4.04619864606906E-06,7.362288603290754E-06,1.3389531296614122E-05,2.4334969898007E-05,4.418854711250906E-05,8.0143574326316E-05,0.0001451206428898359,0.00026221215009896446,0.00047241383951789886,0.0008478506432792254,0.0015138831941066914,0.002684881435418271,0.004719509270093542,0.008200570518363271,0.014038765230316759,0.023584471372180765,0.03870305644084781,0.061731420376382844,0.09521022764755324,0.14132455841667887,0.2011257014658171,0.27381146423732217,0.35645686765475887,0.4444444444444444,0.5324753721790979,0.6157217698211374,0.6906581119263558,0.7553590647074769,0.8093385186735835,0.8531478202604508,0.8879375545991559,0.9151011352412677,0.9360362808926337,0.9520121551919472,0.9641125870704268,0.973226188530969,0.9800612947124725,0.9851714033257601,0.9889828800631293,0.9918207722346132,0.9939310182946776,0.9954986727466336,0.9966624124653979,0.9975258483793887,0.9981662211419222,0.9986410186556701,0.9989929757831075,0.9992538320264273,0.9994471449720269,0.9995903908987,0.9996965299499467,0.9997751705941217,0.9998334349998252,0.9998766016174585],"marker":{},"line":{},"name":"vector [|0.0; 1.0; 3.0; 0.5; 0.5; 1.0; 3.0|]"},{"type":"scatter","mode":"lines","x":[-6.0,-5.9,-5.8,-5.7,-5.6,-5.5,-5.4,-5.3,-5.2,-5.1,-5.0,-4.9,-4.8,-4.7,-4.6,-4.5,-4.4,-4.3,-4.2,-4.1,-4.0,-3.9,-3.8,-3.6999999999999997,-3.5999999999999996,-3.5,-3.4,-3.3,-3.1999999999999997,-3.0999999999999996,-3.0,-2.9,-2.8,-2.6999999999999997,-2.5999999999999996,-2.5,-2.4,-2.3,-2.1999999999999997,-2.0999999999999996,-2.0,-1.8999999999999995,-1.7999999999999998,-1.7000000000000002,-1.5999999999999996,-1.5,-1.3999999999999995,-1.2999999999999998,-1.1999999999999993,-1.0999999999999996,-1.0,-0.8999999999999995,-0.7999999999999998,-0.6999999999999993,-0.5999999999999996,-0.5,-0.39999999999999947,-0.2999999999999998,-0.1999999999999993,-0.09999999999999964,0.0,0.10000000000000053,0.20000000000000018,0.3000000000000007,0.40000000000000036,0.5,0.6000000000000005,0.7000000000000002,0.8000000000000007,0.9000000000000004,1.0,1.1000000000000005,1.2000000000000002,1.3000000000000007,1.4000000000000004,1.5,1.6000000000000005,1.7000000000000002,1.8000000000000007,1.9000000000000004,2.0,2.0999999999999996,2.200000000000001,2.3000000000000007,2.4000000000000004,2.5,2.5999999999999996,2.700000000000001,2.8000000000000007,2.9000000000000004,3.0,3.0999999999999996,3.200000000000001,3.3000000000000007,3.4000000000000004,3.5,3.6000000000000014,3.700000000000001,3.8000000000000007,3.9000000000000004,4.0,4.100000000000001,4.200000000000001,4.300000000000001,4.4,4.5,4.600000000000001,4.700000000000001,4.800000000000001,4.9,5.0,5.100000000000001,5.200000000000001,5.300000000000001,5.4,5.5,5.600000000000001,5.700000000000001,5.800000000000001,5.9,6.0],"y":[0.00017452781601113375,0.0002027723918698757,0.00023558790665987185,0.00027371409447265804,0.0003180104035079797,0.0003694753698031241,0.0004292691262310302,0.000498739554124598,0.0005794526669717682,0.0006732279109858273,0.0007821791781292582,0.0009087624558415617,0.0010558311871716102,0.0012267005885803776,0.0014252223742224318,0.0016558715694985275,0.0019238473682481523,0.002235190303047207,0.002596918363491261,0.0030171851208386736,0.003505463407757988,0.004072758669051441,0.004731856754053747,0.005497611675809359,0.0063872797285930575,0.007420907346298793,0.008621781210904416,0.010016950390607371,0.011637831702677985,0.013520911045787518,0.015708555098636553,0.018249949468873036,0.021202180973351928,0.024631483019170508,0.0286146636673661,0.03324073529626612,0.03861276187260715,0.044849933155478354,0.05208986231166577,0.060491080732953006,0.07023566574551353,0.08153187512332546,0.09461656493985646,0.1097570178192683,0.1272515857213737,0.14742823050403567,0.17063960553977223,0.19725276149750937,0.22763093275931645,0.2621043392076244,0.3009269038940489,0.3442169307545898,0.3918831015265273,0.443543598710067,0.4984557008939508,0.5554831644287452,0.6131326018207011,0.669679327734767,0.7233740198624503,0.7726836926046282,0.8164965809277261,0.8542301218444455,0.8858213170601962,0.9116240610623697,0.9322628137012277,0.9484895546691847,0.9610723247234371,0.9707236400351573,0.9780639990625463,0.983610532676848,0.9877808979327029,0.9909048138149836,0.9932382743621477,0.9949776136587134,0.996272056245539,0.9972342672404667,0.9979488911118347,0.9984792892472788,0.998872763625826,0.9991645567512052,0.9993808873814648,0.9995412396904882,0.9996600813861941,0.9997481488182239,0.9998134057870027,0.9998617575791983,0.9998975819915767,0.9999241238521387,0.9999437879090093,0.999958356148713,0.9999691489767154,0.9999771447256995,0.9999830682459134,0.9999874565654965,0.9999907075498551,0.9999931159587483,0.99999490016316,0.9999962219404558,0.9999972011405397,0.9999979265516585,0.9999984639504509,0.9999988620658266,0.9999991569972574,0.9999993754880036,0.9999995373500217,0.9999996572604046,0.9999997460922291,0.9999998119004784,0.9999998606524368,0.9999998967687804,0.9999999235244287,0.9999999433455017,0.9999999580293144,0.999999968907351,0.9999999769659987,0.9999999829359921,0.9999999873586719,0.9999999906350737,0.999999993062292,0.9999999948604195,0.999999996192505],"marker":{},"line":{},"name":"vector [|0.0; 1.0; 3.0; 2.0; 0.5; 1.0; 0.0|]"},{"type":"scatter","mode":"lines","x":[-6.0,-5.9,-5.8,-5.7,-5.6,-5.5,-5.4,-5.3,-5.2,-5.1,-5.0,-4.9,-4.8,-4.7,-4.6,-4.5,-4.4,-4.3,-4.2,-4.1,-4.0,-3.9,-3.8,-3.6999999999999997,-3.5999999999999996,-3.5,-3.4,-3.3,-3.1999999999999997,-3.0999999999999996,-3.0,-2.9,-2.8,-2.6999999999999997,-2.5999999999999996,-2.5,-2.4,-2.3,-2.1999999999999997,-2.0999999999999996,-2.0,-1.8999999999999995,-1.7999999999999998,-1.7000000000000002,-1.5999999999999996,-1.5,-1.3999999999999995,-1.2999999999999998,-1.1999999999999993,-1.0999999999999996,-1.0,-0.8999999999999995,-0.7999999999999998,-0.6999999999999993,-0.5999999999999996,-0.5,-0.39999999999999947,-0.2999999999999998,-0.1999999999999993,-0.09999999999999964,0.0,0.10000000000000053,0.20000000000000018,0.3000000000000007,0.40000000000000036,0.5,0.6000000000000005,0.7000000000000002,0.8000000000000007,0.9000000000000004,1.0,1.1000000000000005,1.2000000000000002,1.3000000000000007,1.4000000000000004,1.5,1.6000000000000005,1.7000000000000002,1.8000000000000007,1.9000000000000004,2.0,2.0999999999999996,2.200000000000001,2.3000000000000007,2.4000000000000004,2.5,2.5999999999999996,2.700000000000001,2.8000000000000007,2.9000000000000004,3.0,3.0999999999999996,3.200000000000001,3.3000000000000007,3.4000000000000004,3.5,3.6000000000000014,3.700000000000001,3.8000000000000007,3.9000000000000004,4.0,4.100000000000001,4.200000000000001,4.300000000000001,4.4,4.5,4.600000000000001,4.700000000000001,4.800000000000001,4.9,5.0,5.100000000000001,5.200000000000001,5.300000000000001,5.4,5.5,5.600000000000001,5.700000000000001,5.800000000000001,5.9,6.0],"y":[9.278091264452221E-18,1.6905784488665323E-17,3.0804347663523355E-17,5.61291807934906E-17,1.022740350381926E-16,1.8635544071618255E-16,3.3956174884052284E-16,6.187218384781034E-16,1.1273846744811531E-15,2.0542287623819916E-15,3.743048729533139E-15,6.82027916889048E-15,1.2427358179417765E-14,2.2644121211590637E-14,4.1260274633532357E-14,7.518111143346656E-14,1.369888902949227E-13,2.4960996780713004E-13,4.548188561080165E-13,8.28733597414245E-13,1.5100501064819517E-12,2.7514883227076445E-12,5.0135327828301106E-12,9.135238027492764E-12,1.6645453753023332E-11,3.032990763954151E-11,5.5264481960764886E-11,1.0069792777278733E-10,1.8348229902252184E-10,3.3432337784005575E-10,6.091691184407749E-10,1.1099593342738655E-09,2.0224306238185357E-09,3.6849929044176187E-09,6.7142096562737104E-09,1.2233386232496512E-08,2.2288958025934136E-08,4.060888717244023E-08,7.398378435648297E-08,1.3478159115426764E-07,2.4552499554396774E-07,4.4722063950490273E-07,8.145079553810138E-07,1.4831934802314546E-06,2.7002531839757336E-06,4.914529706024728E-06,8.940982469970224E-06,1.6257499561956506E-05,2.9539696392183966E-05,5.362072290557155E-05,9.720463568162153E-05,0.00017590280329930628,0.00031756191693215244,0.0005714873611012265,0.0010241155286357855,0.00182496633365232,0.0032280913285668345,0.005654918194724554,0.009782427937109187,0.016652357272326664,0.027777777777777776,0.04519064623513052,0.07133699684548414,0.10871729160697281,0.15923843387501208,0.2234148448873689,0.2997528289072503,0.38469621475844956,0.47327796660280114,0.5602336148744039,0.6410916043810946,0.7128392235041343,0.7740525990924401,0.8246317579100374,0.8653682181726704,0.8975249050506772,0.9225171491209845,0.9417111643295006,0.9563193514261787,0.9673616609069339,0.9756657954133794,0.9818867924938572,0.9865338665805787,0.9899978095046242,0.9925757390906157,0.9944920146691215,0.99591520711082,0.996971504862875,0.9977551134794589,0.9983362208935118,0.9987670432685739,0.9990863841826142,0.9993230565184464,0.9994984419287047,0.9996284004099313,0.9997246923652677,0.9997960361721251,0.9998488938887506,0.9998880545512885,0.999917066967165,0.9999385607077014,0.9999544841092004,0.9999662807004258,0.9999750199647526,0.9999814942448754,0.9999862905500995,0.9999898437626524,0.9999924760594653,0.9999944261196084,0.9999958707633724,0.9999969409838132,0.9999977338237241,0.999998321174584,0.9999987562951365,0.9999990786405533,0.9999993174400117,0.9999994943470568,0.9999996254030497,0.9999997224917334,0.9999997944168088,0.9999998477002198],"marker":{},"line":{},"name":"vector [|0.0; 1.0; 3.0; 0.5; 5.0; 1.0; 0.0|]"},{"type":"scatter","mode":"lines","x":[-6.0,-5.9,-5.8,-5.7,-5.6,-5.5,-5.4,-5.3,-5.2,-5.1,-5.0,-4.9,-4.8,-4.7,-4.6,-4.5,-4.4,-4.3,-4.2,-4.1,-4.0,-3.9,-3.8,-3.6999999999999997,-3.5999999999999996,-3.5,-3.4,-3.3,-3.1999999999999997,-3.0999999999999996,-3.0,-2.9,-2.8,-2.6999999999999997,-2.5999999999999996,-2.5,-2.4,-2.3,-2.1999999999999997,-2.0999999999999996,-2.0,-1.8999999999999995,-1.7999999999999998,-1.7000000000000002,-1.5999999999999996,-1.5,-1.3999999999999995,-1.2999999999999998,-1.1999999999999993,-1.0999999999999996,-1.0,-0.8999999999999995,-0.7999999999999998,-0.6999999999999993,-0.5999999999999996,-0.5,-0.39999999999999947,-0.2999999999999998,-0.1999999999999993,-0.09999999999999964,0.0,0.10000000000000053,0.20000000000000018,0.3000000000000007,0.40000000000000036,0.5,0.6000000000000005,0.7000000000000002,0.8000000000000007,0.9000000000000004,1.0,1.1000000000000005,1.2000000000000002,1.3000000000000007,1.4000000000000004,1.5,1.6000000000000005,1.7000000000000002,1.8000000000000007,1.9000000000000004,2.0,2.0999999999999996,2.200000000000001,2.3000000000000007,2.4000000000000004,2.5,2.5999999999999996,2.700000000000001,2.8000000000000007,2.9000000000000004,3.0,3.0999999999999996,3.200000000000001,3.3000000000000007,3.4000000000000004,3.5,3.6000000000000014,3.700000000000001,3.8000000000000007,3.9000000000000004,4.0,4.100000000000001,4.200000000000001,4.300000000000001,4.4,4.5,4.600000000000001,4.700000000000001,4.800000000000001,4.9,5.0,5.100000000000001,5.200000000000001,5.300000000000001,5.4,5.5,5.600000000000001,5.700000000000001,5.800000000000001,5.9,6.0],"y":[9.278088494872065E-16,1.690577767659791E-15,3.0804330908543E-15,5.6129139582894916E-15,1.0227393367650356E-14,1.8635519140672433E-14,3.395611356384278E-14,6.187203302450665E-14,1.1273809648289529E-13,2.0542196381180977E-13,3.743026287492503E-13,6.820223970467895E-13,1.2427222413505929E-12,2.2643787282327807E-12,4.125945330338646E-12,7.517909129826679E-12,1.3698392160056275E-11,2.495977469120814E-11,4.547887979584504E-11,8.286596676304305E-11,1.5098682729857878E-10,2.751041099182529E-10,5.012432839284551E-10,9.132532765026592E-10,1.6638800419171871E-09,3.031354486524891E-09,5.5224241959765705E-09,1.0059897302202176E-08,1.8323897492590068E-08,3.337251138733001E-08,6.076983455896952E-08,1.1063442017773571E-07,2.013546740419458E-07,3.6631682839849226E-07,6.66061654809189E-07,1.210185567700491E-06,2.196639441842871E-06,3.9818641793447125E-06,7.205041267536397E-06,1.3006015134025594E-05,2.340230047796826E-05,4.192912931050229E-05,7.469803860323962E-05,0.00013208458706601107,0.00023128085182117525,0.0003998629848055578,0.0006801803344108643,0.0011335776889364784,0.0018421333589788828,0.0029041317763802386,0.004419208153856533,0.006462024572439687,0.009050307250146345,0.0121208547656404,0.015528481767305424,0.019073111775553493,0.022544671794873895,0.025766271242764736,0.028619981771416697,0.031051248972393756,0.03305785123966942,0.034672508090966656,0.03594619421223587,0.03693556952721242,0.037695094901078065,0.038272974042744094,0.0387096893493518,0.03903805388958653,0.03928401395639728,0.03946772763367642,0.03960465833225475,0.03970655911133492,0.03978230287058254,0.03983855506674679,0.03988030468771224,0.03991127590040971,0.039934243209000825,0.03995127059562702,0.039963891817129754,0.03997324570493099,0.03998017735320546,0.039985313607211546,0.03998911927601833,0.039991938935161626,0.03999402798233485,0.039995575692094526,0.03999672232161646,0.039997571797454105,0.03999820112208313,0.03999866734681236,0.039999012739842986,0.03999926861617813,0.03999945817561264,0.03999959860556453,0.03999970263910847,0.039999779709315106,0.039999836804472076,0.039999879101683504,0.039999910436271664,0.039999933649529265,0.03999995084634647,0.039999963586069165,0.03999997302389177,0.039999980015604876,0.03999998519519453,0.03999998903232956,0.03999999187494947,0.039999993980814295,0.03999999554087742,0.03999999669660067,0.03999999755278155,0.039999998187055956,0.03999999865693801,0.039999999005035196,0.03999999926291194,0.03999999945395173,0.03999999959547749,0.03999999970032235,0.03999999977799334,0.039999999835533415,0.03999999987816017],"marker":{},"line":{},"name":"vector [|0.0; 1.0; 3.0; 0.5; 0.5; 5.0; 0.0|]"}];
    var layout = {"width":600,"height":600,"template":{"layout":{"paper_bgcolor":"white","plot_bgcolor":"white","xaxis":{"ticks":"inside","mirror":"all","showline":true,"zeroline":true},"yaxis":{"ticks":"inside","mirror":"all","showline":true,"zeroline":true}},"data":{}},"title":{"text":"Richards Generic"}};
    var config = {"responsive":true};
    Plotly.newPlot('77d94404-95ab-4531-bf02-71e4a93567ed', data, layout, config);
};
renderPlotly_77d9440495ab4531bf0271e4a93567ed();
</script></div>

<div><div id="93e609de-2658-4d23-a589-3c8fda0f0f3e"><!-- Plotly chart will be drawn inside this DIV --></div><script type="text/javascript">var renderPlotly_93e609de26584d23a5893c8fda0f0f3e = function() {
    var data = [{"type":"scatter","mode":"lines","x":[0.0,0.1,0.2,0.30000000000000004,0.4,0.5,0.6000000000000001,0.7000000000000001,0.8,0.9,1.0,1.1,1.2000000000000002,1.3,1.4000000000000001,1.5,1.6,1.7000000000000002,1.8,1.9000000000000001,2.0,2.1,2.2,2.3000000000000003,2.4000000000000004,2.5,2.6,2.7,2.8000000000000003,2.9000000000000004,3.0,3.1,3.2,3.3000000000000003,3.4000000000000004,3.5,3.6,3.7,3.8000000000000003,3.9000000000000004,4.0,4.1000000000000005,4.2,4.3,4.4,4.5,4.6000000000000005,4.7,4.800000000000001,4.9,5.0,5.1000000000000005,5.2,5.300000000000001,5.4,5.5,5.6000000000000005,5.7,5.800000000000001,5.9,6.0,6.1000000000000005,6.2,6.300000000000001,6.4,6.5,6.6000000000000005,6.7,6.800000000000001,6.9,7.0,7.1000000000000005,7.2,7.300000000000001,7.4,7.5,7.6000000000000005,7.7,7.800000000000001,7.9,8.0,8.1,8.200000000000001,8.3,8.4,8.5,8.6,8.700000000000001,8.8,8.9,9.0,9.1,9.200000000000001,9.3,9.4,9.5,9.600000000000001,9.700000000000001,9.8,9.9,10.0],"y":[7.0,7.000103999584001,7.0008319733765685,7.002807696757834,7.006654296354769,7.012993502166125,7.022444602278696,7.035623102750909,7.053139096835858,7.0755953496965756,7.103585107118212,7.137689639425648,7.178475534845049,7.226491759871124,7.282266507777239,7.346303860183621,7.419080290521514,7.5010410422308205,7.592596418530233,7.694118024517332,7.805935006100517,7.928330333736774,8.061537182046253,8.205735458990452,8.361048540328525,8.527540266400258,8.705212258823526,8.894001614344484,9.09377903175718,9.30434742544575,9.525441075644766,9.756725360929078,9.9977971127251,10.248185624797753,10.507354342756726,10.77470324971729,11.04957195445019,11.331243477798811,11.618948721995636,11.911871595968403,12.209154758008001,12.50990592551548,12.813204690210107,13.118109766431054,13.423666590270628,13.728915178503364,14.03289814887562,14.334668797527975,14.633299125343783,14.927887703020284,15.21756726477125,15.501511922869332,15.778943899746377,16.049139681057298,16.31143550188499,16.56523208896988,16.809998594279033,17.045275669127477,17.270677643120937,17.485893788060622,17.69068866326325,17.884901555111593,18.06844503966551,18.241302712438536,18.40352614360881,18.555231129641914,18.696593323257236,18.827843332618187,18.94926138738001,19.06117167366083,19.163936442055988,19.257949992503924,19.3436326372163,19.421424738149657,19.49178090882504,19.55516446194834,19.61204217454139,19.662879431484466,19.708135796834313,19.74826105036274,19.783691714797595,19.81484808756021,19.84213177967426,19.865923754226337,19.886582847502904,19.904444747877537,19.919821400794156,19.933000802852657,19.944247144071586,19.953801254850912,19.96188131291948,19.968683765523767,19.974384423167976,19.979139683199463,19.98308784428516,19.9863504761745,19.989033812924347,19.991230141809233,19.993019164306684,19.99446930969729,19.995638985837267],"marker":{},"line":{},"name":"vector [|7.0; 20.0; 0.2; 3.0|]"},{"type":"scatter","mode":"lines","x":[0.0,0.1,0.2,0.30000000000000004,0.4,0.5,0.6000000000000001,0.7000000000000001,0.8,0.9,1.0,1.1,1.2000000000000002,1.3,1.4000000000000001,1.5,1.6,1.7000000000000002,1.8,1.9000000000000001,2.0,2.1,2.2,2.3000000000000003,2.4000000000000004,2.5,2.6,2.7,2.8000000000000003,2.9000000000000004,3.0,3.1,3.2,3.3000000000000003,3.4000000000000004,3.5,3.6,3.7,3.8000000000000003,3.9000000000000004,4.0,4.1000000000000005,4.2,4.3,4.4,4.5,4.6000000000000005,4.7,4.800000000000001,4.9,5.0,5.1000000000000005,5.2,5.300000000000001,5.4,5.5,5.6000000000000005,5.7,5.800000000000001,5.9,6.0,6.1000000000000005,6.2,6.300000000000001,6.4,6.5,6.6000000000000005,6.7,6.800000000000001,6.9,7.0,7.1000000000000005,7.2,7.300000000000001,7.4,7.5,7.6000000000000005,7.7,7.800000000000001,7.9,8.0,8.1,8.200000000000001,8.3,8.4,8.5,8.6,8.700000000000001,8.8,8.9,9.0,9.1,9.200000000000001,9.3,9.4,9.5,9.600000000000001,9.700000000000001,9.8,9.9,10.0],"y":[7.0,7.000000315899996,7.000010108796069,7.000076763473359,7.000323477575405,7.000987150018673,7.002456206334248,7.00530824725532,7.010347291073877,7.018640202575128,7.031551649220422,7.050776588006711,7.078368856771691,7.116763920055018,7.168793209464052,7.237686822862107,7.327060645715614,7.440883297141234,7.583417779987201,7.759132463206322,7.972576219588786,8.228213391578363,8.530215992744932,8.882213396507613,9.287003889020603,9.74623792490297,10.260089578298988,10.826940092463929,11.44310479158392,12.10264070797145,12.797275498285572,13.516496757245964,14.247832962699741,14.977341839715141,15.690298887507232,16.37204983855071,17.00895961819535,17.589362579509118,18.10440120569554,18.548639602505766,18.920358158967534,19.221476675113585,19.45710946921599,19.634816680924118,19.76366733894432,19.85325869611278,19.91283487400739,19.950616272537093,19.97339848380318,19.986420570842135,19.99345334235342,19.997029913618288,19.9987366756004,19.999498145902244,19.999814556578112,19.999936524779404,19.999979961052425,19.999994191380527,19.99999846121914,19.999999629240083,19.99999991915488,19.9999999841285,19.999999997209592,19.999999999563073,19.999999999939416,19.999999999992603,19.99999999999921,19.999999999999925,19.999999999999993,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0],"marker":{},"line":{},"name":"vector [|7.0; 20.0; 0.3; 5.0|]"},{"type":"scatter","mode":"lines","x":[0.0,0.1,0.2,0.30000000000000004,0.4,0.5,0.6000000000000001,0.7000000000000001,0.8,0.9,1.0,1.1,1.2000000000000002,1.3,1.4000000000000001,1.5,1.6,1.7000000000000002,1.8,1.9000000000000001,2.0,2.1,2.2,2.3000000000000003,2.4000000000000004,2.5,2.6,2.7,2.8000000000000003,2.9000000000000004,3.0,3.1,3.2,3.3000000000000003,3.4000000000000004,3.5,3.6,3.7,3.8000000000000003,3.9000000000000004,4.0,4.1000000000000005,4.2,4.3,4.4,4.5,4.6000000000000005,4.7,4.800000000000001,4.9,5.0,5.1000000000000005,5.2,5.300000000000001,5.4,5.5,5.6000000000000005,5.7,5.800000000000001,5.9,6.0,6.1000000000000005,6.2,6.300000000000001,6.4,6.5,6.6000000000000005,6.7,6.800000000000001,6.9,7.0,7.1000000000000005,7.2,7.300000000000001,7.4,7.5,7.6000000000000005,7.7,7.800000000000001,7.9,8.0,8.1,8.200000000000001,8.3,8.4,8.5,8.6,8.700000000000001,8.8,8.9,9.0,9.1,9.200000000000001,9.3,9.4,9.5,9.600000000000001,9.700000000000001,9.8,9.9,10.0],"y":[7.0,7.000063999744,7.00051198361635,7.0017278133894365,7.0040949516029345,7.007996001333,7.013812062940736,7.021921909385175,7.032700982668221,7.0465202151978925,7.063744681303515,7.084732085800399,7.109831098366184,7.139379544536077,7.173702466324454,7.213110067805306,7.257895563397855,7.30833294906512,7.364674719095528,7.427149553549127,7.495960003754164,7.5712802053764765,7.653253650490002,7.741991051686432,7.8375683325098615,7.940024779323236,8.049361390045247,8.165539454981221,8.288479404158263,8.418059954120462,8.554117585012165,8.696446375956356,8.844798223215447,8.998883461414001,9.158371903234908,9.322894307518332,9.492044279661656,9.665380601722344,9.842429982766546,10.022690212903633,10.205633697235694,10.390711338778756,10.57735673243699,10.764990625496033,10.953025594012693,11.140870879078992,11.327937322384997,11.513642336940293,11.697414846365405,11.878700124935559,12.056964470628461,12.231699644842665,12.40242701522854,12.568701342189108,12.730114155006149,12.88629667013531,13.03692221186402,13.181708104078448,13.320417011151346,13.452857715729614,13.578885331238922,13.69840095699175,13.811350793640312,13.917724746116022,14.017554549913116,14.110911464395025,14.197903583542914,14.278672820072732,14.353391623003082,14.422259491483587,14.48549934895753,14.543353841540878,14.59608162290234,14.643953685015173,14.68724979004618,14.726255053506671,14.761256722794702,14.792541188605826,14.820391259590346,14.845083723300148,14.866887209106212,14.886060361575515,14.902850325953391,14.917491541062361,14.930204829232556,14.94119676792464,14.950659323565635,14.958769724832406,14.965690550197898,14.971570002985176,14.976542346411987,14.980728471091549,14.98423656810337,14.987162881968901,14.989592519560098,14.991600293030462,14.993251577184212,14.994603164190297,14.995704101111805,14.996596498275256,14.99731629897678],"marker":{},"line":{},"name":"vector [|7.0; 15.0; 0.2; 3.0|]"}];
    var layout = {"width":600,"height":600,"template":{"layout":{"paper_bgcolor":"white","plot_bgcolor":"white","xaxis":{"ticks":"inside","mirror":"all","showline":true,"zeroline":true},"yaxis":{"ticks":"inside","mirror":"all","showline":true,"zeroline":true}},"data":{}},"title":{"text":"Weibull"}};
    var config = {"responsive":true};
    Plotly.newPlot('93e609de-2658-4d23-a589-3c8fda0f0f3e', data, layout, config);
};
renderPlotly_93e609de26584d23a5893c8fda0f0f3e();
</script></div>

<div><div id="fadd3e4e-b1c1-4c5e-91bb-9c9c2e765508"><!-- Plotly chart will be drawn inside this DIV --></div><script type="text/javascript">var renderPlotly_fadd3e4eb1c14c5e91bb9c9c2e765508 = function() {
    var data = [{"type":"scatter","mode":"lines","x":[0.0,0.1,0.2,0.30000000000000004,0.4,0.5,0.6000000000000001,0.7000000000000001,0.8,0.9,1.0,1.1,1.2000000000000002,1.3,1.4000000000000001,1.5,1.6,1.7000000000000002,1.8,1.9000000000000001,2.0,2.1,2.2,2.3000000000000003,2.4000000000000004,2.5,2.6,2.7,2.8000000000000003,2.9000000000000004,3.0,3.1,3.2,3.3000000000000003,3.4000000000000004,3.5,3.6,3.7,3.8000000000000003,3.9000000000000004,4.0,4.1000000000000005,4.2,4.3,4.4,4.5,4.6000000000000005,4.7,4.800000000000001,4.9,5.0,5.1000000000000005,5.2,5.300000000000001,5.4,5.5,5.6000000000000005,5.7,5.800000000000001,5.9,6.0,6.1000000000000005,6.2,6.300000000000001,6.4,6.5,6.6000000000000005,6.7,6.800000000000001,6.9,7.0,7.1000000000000005,7.2,7.300000000000001,7.4,7.5,7.6000000000000005,7.7,7.800000000000001,7.9,8.0,8.1,8.200000000000001,8.3,8.4,8.5,8.6,8.700000000000001,8.8,8.9,9.0,9.1,9.200000000000001,9.3,9.4,9.5,9.600000000000001,9.700000000000001,9.8,9.9,10.0],"y":[7.0,7.000259997400017,7.0020798336088745,7.007018104941126,7.016629354942376,7.032459408833018,7.056038868891573,7.088874810864587,7.132440746097107,7.188164944282452,7.257417247012182,7.341494541937898,7.4416051103745495,7.558852106370393,7.694216471179349,7.848539631991642,8.022506375784488,8.216628326146711,8.431228480695147,8.666427286985815,8.922130743439252,9.198021006727949,9.493549966589363,9.807936211825805,10.14016575657537,10.488996823693657,10.852968892945587,11.230416117178574,11.6194850921134,12.0181568380942,12.424272719138134,12.835563890669793,13.249683737695793,13.664242645466196,14.076844340527217,14.485122956899588,14.886779924771398,15.279619751565782,15.661583770376101,16.030780970007665,16.385515094108477,16.72430730232793,17.045913820660097,17.34933816608967,17.633837705986,17.89892449795059,18.144360542907275,18.370147764794044,18.57651319617447,18.763889993050267,18.932895017889315,19.084303812664693,19.219023830598708,19.33806680533355,19.442521110391816,19.533524902898385,19.612240758106296,19.67983239111245,19.737443935980707,19.78618211747927,19.827101513948232,19.8611929781565,19.88937516227397,19.912488988037616,19.931294817267823,19.94647201311633,19.958620539420938,19.96826422363545,19.97585530622624,19.981779913552966,19.986364118875837,19.98988029375318,19.9925534961792,19.99456768902981,19.996071629784456,19.9971843176321,19.99799992508683,19.99859217686367,19.999018168293432,19.999321638782376,19.99953573295466,19.999685293670893,19.99978873782126,19.99985956848736,19.999907576625684,19.999939782686212,19.999961164300437,19.999975211002255,19.999984341405817,19.9999902127559,19.99999394756957,19.999996297382822,19.999997759498072,19.999998659128313,19.999999206435206,19.99999953561442,19.999999731326955,19.999999846338028,19.999999913133262,19.999999951467426,19.999999973205004],"marker":{},"line":{},"name":"vector [|7.0; 20.0; 0.02; 3.0|]"},{"type":"scatter","mode":"lines","x":[0.0,0.1,0.2,0.30000000000000004,0.4,0.5,0.6000000000000001,0.7000000000000001,0.8,0.9,1.0,1.1,1.2000000000000002,1.3,1.4000000000000001,1.5,1.6,1.7000000000000002,1.8,1.9000000000000001,2.0,2.1,2.2,2.3000000000000003,2.4000000000000004,2.5,2.6,2.7,2.8000000000000003,2.9000000000000004,3.0,3.1,3.2,3.3000000000000003,3.4000000000000004,3.5,3.6,3.7,3.8000000000000003,3.9000000000000004,4.0,4.1000000000000005,4.2,4.3,4.4,4.5,4.6000000000000005,4.7,4.800000000000001,4.9,5.0,5.1000000000000005,5.2,5.300000000000001,5.4,5.5,5.6000000000000005,5.7,5.800000000000001,5.9,6.0,6.1000000000000005,6.2,6.300000000000001,6.4,6.5,6.6000000000000005,6.7,6.800000000000001,6.9,7.0,7.1000000000000005,7.2,7.300000000000001,7.4,7.5,7.6000000000000005,7.7,7.800000000000001,7.9,8.0,8.1,8.200000000000001,8.3,8.4,8.5,8.6,8.700000000000001,8.8,8.9,9.0,9.1,9.200000000000001,9.3,9.4,9.5,9.600000000000001,9.700000000000001,9.8,9.9,10.0],"y":[7.0,7.0,7.000000000012779,7.000000005596075,7.000000418759305,7.000011901850019,7.000183370850708,7.0018514171399975,7.013714665712776,7.0800500630394065,7.384208063869394,8.531181564339882,11.811434661108947,17.200691095096595,19.877799667252393,19.999974373084036,19.99999999999999,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0],"marker":{},"line":{},"name":"vector [|7.0; 20.0; 0.03; 15.0|]"},{"type":"scatter","mode":"lines","x":[0.0,0.1,0.2,0.30000000000000004,0.4,0.5,0.6000000000000001,0.7000000000000001,0.8,0.9,1.0,1.1,1.2000000000000002,1.3,1.4000000000000001,1.5,1.6,1.7000000000000002,1.8,1.9000000000000001,2.0,2.1,2.2,2.3000000000000003,2.4000000000000004,2.5,2.6,2.7,2.8000000000000003,2.9000000000000004,3.0,3.1,3.2,3.3000000000000003,3.4000000000000004,3.5,3.6,3.7,3.8000000000000003,3.9000000000000004,4.0,4.1000000000000005,4.2,4.3,4.4,4.5,4.6000000000000005,4.7,4.800000000000001,4.9,5.0,5.1000000000000005,5.2,5.300000000000001,5.4,5.5,5.6000000000000005,5.7,5.800000000000001,5.9,6.0,6.1000000000000005,6.2,6.300000000000001,6.4,6.5,6.6000000000000005,6.7,6.800000000000001,6.9,7.0,7.1000000000000005,7.2,7.300000000000001,7.4,7.5,7.6000000000000005,7.7,7.800000000000001,7.9,8.0,8.1,8.200000000000001,8.3,8.4,8.5,8.6,8.700000000000001,8.8,8.9,9.0,9.1,9.200000000000001,9.3,9.4,9.5,9.600000000000001,9.700000000000001,9.8,9.9,10.0],"y":[7.0,7.000159998400011,7.001279897605461,7.004318833809924,7.010233449195308,7.019975020820319,7.03448545777943,7.054692191301284,7.081501997598219,7.115793811866125,7.158410613545958,7.210150487346398,7.271756990999723,7.343908988535627,7.427210136110369,7.52217823507178,7.629234692790454,7.748694354551823,7.88075598812009,8.025493715068194,8.18284968827031,8.352628311832584,8.534492287131915,8.727960745738956,8.932409696354075,9.147074968426866,9.371057780274207,9.603332995186815,9.842760056685169,10.088096515750276,10.338013981008082,10.59111624041218,10.845959223197411,11.101072397209967,11.354981132632133,11.606229511938206,11.853403030628552,12.095150616348175,12.330205397154524,12.557403673850871,12.775701596374446,12.984189109124879,13.182100812713907,13.368823486824413,13.543900126760615,13.707030460277286,13.858068026404478,13.997014009104028,14.12400812072275,14.239316918800165,14.34332001100881,14.436494653947504,14.519399280368436,14.592656495589878,14.656936067933424,14.712938401783623,14.761378928065412,14.802973779146122,14.838427037526587,14.868419764602628,14.89360093166045,14.914580294250154,14.931923176783982,14.94614706956161,14.957719887549429,14.96705970037928,14.97453571656673,14.98047029146797,14.985141726908456,14.988787639109518,14.991608688538978,14.993772488463497,14.995417536110278,14.99665703940296,14.99758254140582,14.998267272388984,14.998769184668818,14.99913364730072,14.999395795872882,14.999582546943001,14.999714297202868,14.999806334566703,14.99986999250539,14.999913580607604,14.999943124077346,14.999962943191514,14.999976101107961,14.999984745232156,14.99999036394204,14.999993977080553,14.999996275427428,14.99999772146635,14.999998621229583,14.999999174848192,14.999999511652433,14.99999971422426,14.999999834662741,14.999999905438788,14.999999946543547,14.999999970133802,14.99999998351077],"marker":{},"line":{},"name":"vector [|7.0; 15.0; 0.02; 3.0|]"}];
    var layout = {"width":600,"height":600,"template":{"layout":{"paper_bgcolor":"white","plot_bgcolor":"white","xaxis":{"ticks":"inside","mirror":"all","showline":true,"zeroline":true},"yaxis":{"ticks":"inside","mirror":"all","showline":true,"zeroline":true}},"data":{}},"title":{"text":"Janoschek"}};
    var config = {"responsive":true};
    Plotly.newPlot('fadd3e4e-b1c1-4c5e-91bb-9c9c2e765508', data, layout, config);
};
renderPlotly_fadd3e4eb1c14c5e91bb9c9c2e765508();
</script></div>

<div><div id="bfbd302b-a631-423f-9006-4e0751e43017"><!-- Plotly chart will be drawn inside this DIV --></div><script type="text/javascript">var renderPlotly_bfbd302ba631423f90064e0751e43017 = function() {
    var data = [{"type":"scatter","mode":"lines","x":[0.0,0.1,0.2,0.30000000000000004,0.4,0.5,0.6000000000000001,0.7000000000000001,0.8,0.9,1.0,1.1,1.2000000000000002,1.3,1.4000000000000001,1.5,1.6,1.7000000000000002,1.8,1.9000000000000001,2.0,2.1,2.2,2.3000000000000003,2.4000000000000004,2.5,2.6,2.7,2.8000000000000003,2.9000000000000004,3.0,3.1,3.2,3.3000000000000003,3.4000000000000004,3.5,3.6,3.7,3.8000000000000003,3.9000000000000004,4.0,4.1000000000000005,4.2,4.3,4.4,4.5,4.6000000000000005,4.7,4.800000000000001,4.9,5.0,5.1000000000000005,5.2,5.300000000000001,5.4,5.5,5.6000000000000005,5.7,5.800000000000001,5.9,6.0,6.1000000000000005,6.2,6.300000000000001,6.4,6.5,6.6000000000000005,6.7,6.800000000000001,6.9,7.0,7.1000000000000005,7.2,7.300000000000001,7.4,7.5,7.6000000000000005,7.7,7.800000000000001,7.9,8.0,8.1,8.200000000000001,8.3,8.4,8.5,8.6,8.700000000000001,8.8,8.9,9.0,9.1,9.200000000000001,9.3,9.4,9.5,9.600000000000001,9.700000000000001,9.8,9.9,10.0],"y":[7.0,8.237113565532526,9.356500209986237,10.369363131137668,11.28583940153669,12.115101423735766,12.865448730777658,13.544391050711678,14.15872346647612,14.714594423372212,15.21756726477125,15.672675911924966,16.084475245141373,16.457086690557837,16.794239468759116,17.099307918070412,17.37534526606948,17.625114187314452,17.851114453119376,18.055607950105745,18.240641317924034,18.408066432711234,18.55955894128966,18.696635031603552,18.820666607237637,18.932895017889315,19.03444348321366,19.126328334383253,19.209469185872166,19.284698139266705,19.35276811121777,19.41436036888375,19.47009134828124,19.52051882378388,19.56614749051576,19.60743401550986,19.644791608185198,19.678594155885587,19.709179965869847,19.736855151204544,19.761896694446456,19.784555219777104,19.80505750133379,19.82360873284139,19.84039458126011,19.85558304500285,19.869326135319763,19.881761397677955,19.893013288362738,19.903194420077984,19.91240668901189,19.920742294648296,19.92828466253011,19.935109279210167,19.941284447746035,19.946871971299966,19.95192777168572,19.956502449052874,19.960641788310113,19.964387217356013,19.96777622170334,19.970842719646683,19.973617401728156,19.976128037898626,19.978399755448738,19.980455290491292,19.98231521551188,19.983998145265243,19.985520923078017,19.98689878942237,19.988145534447792,19.98927363599754,19.990294384491104,19.99121799592248,19.992053714105314,19.99280990318808,19.99349413136527,19.994113246622472,19.994673445273264,19.99518033397403,19.995638985837267,19.996053991204974,19.996429503590363,19.996769281247598,19.997076724785675,19.997354911202862,19.997606624682323,19.99783438445716,19.99804047002376,19.998226943955732,19.998395672546874,19.998548344489627,19.99868648777612,19.998811484990785,19.998924587147762,19.99902692621146,19.99911952642562,19.99920331456431,19.99927912920738,19.99934772913327,19.99940980091309],"marker":{},"line":{},"name":"vector [|7.0; 20.0; 1.0|]"},{"type":"scatter","mode":"lines","x":[0.0,0.1,0.2,0.30000000000000004,0.4,0.5,0.6000000000000001,0.7000000000000001,0.8,0.9,1.0,1.1,1.2000000000000002,1.3,1.4000000000000001,1.5,1.6,1.7000000000000002,1.8,1.9000000000000001,2.0,2.1,2.2,2.3000000000000003,2.4000000000000004,2.5,2.6,2.7,2.8000000000000003,2.9000000000000004,3.0,3.1,3.2,3.3000000000000003,3.4000000000000004,3.5,3.6,3.7,3.8000000000000003,3.9000000000000004,4.0,4.1000000000000005,4.2,4.3,4.4,4.5,4.6000000000000005,4.7,4.800000000000001,4.9,5.0,5.1000000000000005,5.2,5.300000000000001,5.4,5.5,5.6000000000000005,5.7,5.800000000000001,5.9,6.0,6.1000000000000005,6.2,6.300000000000001,6.4,6.5,6.6000000000000005,6.7,6.800000000000001,6.9,7.0,7.1000000000000005,7.2,7.300000000000001,7.4,7.5,7.6000000000000005,7.7,7.800000000000001,7.9,8.0,8.1,8.200000000000001,8.3,8.4,8.5,8.6,8.700000000000001,8.8,8.9,9.0,9.1,9.200000000000001,9.3,9.4,9.5,9.600000000000001,9.700000000000001,9.8,9.9,10.0],"y":[7.0,9.356500209986237,11.28583940153669,12.865448730777658,14.15872346647612,15.21756726477125,16.084475245141373,16.794239468759116,17.37534526606948,17.851114453119376,18.240641317924034,18.55955894128966,18.820666607237637,19.03444348321366,19.209469185872166,19.35276811121777,19.47009134828124,19.56614749051576,19.644791608185198,19.709179965869847,19.761896694446456,19.80505750133379,19.84039458126011,19.869326135319763,19.893013288362738,19.91240668901189,19.92828466253011,19.941284447746035,19.95192777168572,19.960641788310113,19.96777622170334,19.973617401728156,19.978399755448738,19.98231521551188,19.985520923078017,19.988145534447792,19.990294384491104,19.992053714105314,19.99349413136527,19.994673445273264,19.995638985837267,19.996429503590363,19.997076724785675,19.997606624682323,19.99804047002376,19.998395672546874,19.99868648777612,19.998924587147762,19.99911952642562,19.99927912920738,19.99940980091309,19.999516785857107,19.999604377720893,19.99967609187349,19.999734806455656,19.999782877889725,19.999822235451145,19.999854458697047,19.99988084085943,19.999902440747103,19.999920125239406,19.999934604077108,19.999946458346802,19.999956163801958,19.999964109956565,19.99997061571771,19.99997594218443,19.99998030312654,19.99998387356396,19.99998679679088,19.99998919012665,19.999991149624254,19.9999927539252,19.99999406741572,19.999995142810807,19.999996023269834,19.999996744128715,19.999997334318053,19.99999781752421,19.999998213139953,19.99999853704273,19.999998802231893,19.999999019350415,19.999999197112025,19.999999342650923,19.9999994618081,19.999999559365737,19.999999639239178,19.99999970463402,19.99999975817479,19.99999980201026,19.999999837899715,19.99999986728351,19.999999891340927,19.999999911037477,19.999999927163646,19.99999994036664,19.99999995117633,19.99999996002656,19.999999967272515,19.999999973205004],"marker":{},"line":{},"name":"vector [|7.0; 20.0; 2.0|]"},{"type":"scatter","mode":"lines","x":[0.0,0.1,0.2,0.30000000000000004,0.4,0.5,0.6000000000000001,0.7000000000000001,0.8,0.9,1.0,1.1,1.2000000000000002,1.3,1.4000000000000001,1.5,1.6,1.7000000000000002,1.8,1.9000000000000001,2.0,2.1,2.2,2.3000000000000003,2.4000000000000004,2.5,2.6,2.7,2.8000000000000003,2.9000000000000004,3.0,3.1,3.2,3.3000000000000003,3.4000000000000004,3.5,3.6,3.7,3.8000000000000003,3.9000000000000004,4.0,4.1000000000000005,4.2,4.3,4.4,4.5,4.6000000000000005,4.7,4.800000000000001,4.9,5.0,5.1000000000000005,5.2,5.300000000000001,5.4,5.5,5.6000000000000005,5.7,5.800000000000001,5.9,6.0,6.1000000000000005,6.2,6.300000000000001,6.4,6.5,6.6000000000000005,6.7,6.800000000000001,6.9,7.0,7.1000000000000005,7.2,7.300000000000001,7.4,7.5,7.6000000000000005,7.7,7.800000000000001,7.9,8.0,8.1,8.200000000000001,8.3,8.4,8.5,8.6,8.700000000000001,8.8,8.9,9.0,9.1,9.200000000000001,9.3,9.4,9.5,9.600000000000001,9.700000000000001,9.8,9.9,10.0],"y":[7.0,9.073454234546258,10.609506911247788,11.747442722075208,12.590446304702384,13.21495871881256,13.677608894227308,14.020348573976145,14.2742563736847,14.462355898082002,14.60170345305709,14.70493466079008,14.781410220421659,14.838064708433565,14.880035385436178,14.911128027694062,14.93416202360784,14.951226027475874,14.963867352459099,14.97323227634023,14.98016998258667,14.985309561783769,14.989117055699618,14.991937716567612,14.994027313532987,14.995575325038818,14.996722120168162,14.99757168689537,14.998201061406569,14.9986673135121,14.999012721567306,14.999268606148174,14.999458170108074,14.99959860254355,14.999702637450527,14.999779708405201,14.99983680397271,14.99987910140944,14.99991043612126,14.999933649446714,14.999950846301173,14.999963586044295,14.999973023878127,14.999980015597393,14.999985195190419,14.999989032327308,14.999991874948233,14.999993980813606,14.999995540877046,14.999996696600467,14.999997552781435,14.999998187055898,14.999998656937976,14.999999005035182,14.999999262911933,14.99999945395173,14.999999595477492,14.999999700322356,14.99999977799334,14.999999835533421,14.999999878160162,14.999999909738827,14.999999933132878,14.999999950463618,14.999999963302546,14.999999972813857,14.99999997986001,14.99999998507993,14.99999998894694,14.999999991811691,14.999999993933951,14.99999999550616,14.999999996670882,14.99999999753373,14.99999999817294,14.999999998646482,14.99999999899729,14.999999999257174,14.9999999994497,14.999999999592328,14.99999999969799,14.999999999776264,14.999999999834253,14.999999999877211,14.999999999909036,14.999999999932612,14.999999999950077,14.999999999963016,14.999999999972601,14.999999999979703,14.999999999984963,14.99999999998886,14.999999999991749,14.999999999993888,14.99999999999547,14.999999999996644,14.999999999997515,14.999999999998158,14.999999999998636,14.99999999999899,14.999999999999252],"marker":{},"line":{},"name":"vector [|7.0; 15.0; 3.0|]"}];
    var layout = {"width":600,"height":600,"template":{"layout":{"paper_bgcolor":"white","plot_bgcolor":"white","xaxis":{"ticks":"inside","mirror":"all","showline":true,"zeroline":true},"yaxis":{"ticks":"inside","mirror":"all","showline":true,"zeroline":true}},"data":{}},"title":{"text":"Exponential"}};
    var config = {"responsive":true};
    Plotly.newPlot('bfbd302b-a631-423f-9006-4e0751e43017', data, layout, config);
};
renderPlotly_bfbd302ba631423f90064e0751e43017();
</script></div>

<div><div id="f4084ece-17f3-4612-9bd9-b0d625f0bb10"><!-- Plotly chart will be drawn inside this DIV --></div><script type="text/javascript">var renderPlotly_f4084ece17f346129bd9b0d625f0bb10 = function() {
    var data = [{"type":"scatter","mode":"lines","x":[0.0,0.1,0.2,0.30000000000000004,0.4,0.5,0.6000000000000001,0.7000000000000001,0.8,0.9,1.0,1.1,1.2000000000000002,1.3,1.4000000000000001,1.5,1.6,1.7000000000000002,1.8,1.9000000000000001,2.0,2.1,2.2,2.3000000000000003,2.4000000000000004,2.5,2.6,2.7,2.8000000000000003,2.9000000000000004,3.0,3.1,3.2,3.3000000000000003,3.4000000000000004,3.5,3.6,3.7,3.8000000000000003,3.9000000000000004,4.0,4.1000000000000005,4.2,4.3,4.4,4.5,4.6000000000000005,4.7,4.800000000000001,4.9,5.0,5.1000000000000005,5.2,5.300000000000001,5.4,5.5,5.6000000000000005,5.7,5.800000000000001,5.9,6.0,6.1000000000000005,6.2,6.300000000000001,6.4,6.5,6.6000000000000005,6.7,6.800000000000001,6.9,7.0,7.1000000000000005,7.2,7.300000000000001,7.4,7.5,7.6000000000000005,7.7,7.800000000000001,7.9,8.0,8.1,8.200000000000001,8.3,8.4,8.5,8.6,8.700000000000001,8.8,8.9,9.0,9.1,9.200000000000001,9.3,9.4,9.5,9.600000000000001,9.700000000000001,9.8,9.9,10.0],"y":[0.5319398715373173,0.7114237854527236,0.9485174635513356,1.2594671211399302,1.6634539298784474,2.18193642391226,2.837021298009757,3.6485104761271283,4.629504330019647,5.781009947499922,7.086873875484092,8.51114966376682,10.000000000000004,11.488850336233181,12.913126124515912,14.21899005250008,15.370495669980354,16.351489523872875,17.162978701990244,17.81806357608774,18.336546070121553,18.74053287886007,19.051482536448667,19.288576214547277,19.468060128462685,19.60319388531845,19.70451936613454,19.78026114738814,19.836748576936802,19.878803970168317,19.910074536781178,19.933303853851733,19.950547536867305,19.963341221150145,19.97282960099142,19.97986458359829,19.985079423323267,19.98894442726153,19.99180865670028,19.993931059399422,19.995503664595333,19.996668838704455,19.997532108480275,19.998171682522955,19.998645517007606,19.998996556706324,19.999256621257942,19.999449286177708,19.9995920182544,19.99969775809118,19.9997760928099,19.99983412499252,19.999877116507957,19.999908965525105,19.999932559922726,19.999950039118286,19.99996298804454,19.99997258085586,19.99997968739121,19.999984952045338,19.99998885219883,19.99999174150458,19.99999388195546,19.999995467640773,19.999996642345504,19.999997512588266,19.999998157280004,19.999998634879418,19.999998988693783,19.999999250805917,19.999999444983366,19.99999958883356,19.999999695400412,19.99999977434707,19.9999998328322,19.999999876159045,19.999999908256367,19.999999932034644,19.99999994965003,19.999999962699825,19.99999997236735,19.99999997952923,19.99999998483488,19.9999999887654,19.999999991677203,19.999999993834322,19.99999999543235,19.999999996616204,19.999999997493223,19.999999998142936,19.99999999862425,19.99999999898082,19.999999999244974,19.99999999944066,19.999999999585633,19.999999999693028,19.99999999977259,19.99999999983153,19.999999999875193,19.99999999990754,19.999999999931504],"marker":{},"line":{},"name":"vector [|20.0; 3.0; 1.2|]"},{"type":"scatter","mode":"lines","x":[0.0,0.1,0.2,0.30000000000000004,0.4,0.5,0.6000000000000001,0.7000000000000001,0.8,0.9,1.0,1.1,1.2000000000000002,1.3,1.4000000000000001,1.5,1.6,1.7000000000000002,1.8,1.9000000000000001,2.0,2.1,2.2,2.3000000000000003,2.4000000000000004,2.5,2.6,2.7,2.8000000000000003,2.9000000000000004,3.0,3.1,3.2,3.3000000000000003,3.4000000000000004,3.5,3.6,3.7,3.8000000000000003,3.9000000000000004,4.0,4.1000000000000005,4.2,4.3,4.4,4.5,4.6000000000000005,4.7,4.800000000000001,4.9,5.0,5.1000000000000005,5.2,5.300000000000001,5.4,5.5,5.6000000000000005,5.7,5.800000000000001,5.9,6.0,6.1000000000000005,6.2,6.300000000000001,6.4,6.5,6.6000000000000005,6.7,6.800000000000001,6.9,7.0,7.1000000000000005,7.2,7.300000000000001,7.4,7.5,7.6000000000000005,7.7,7.800000000000001,7.9,8.0,8.1,8.200000000000001,8.3,8.4,8.5,8.6,8.700000000000001,8.8,8.9,9.0,9.1,9.200000000000001,9.3,9.4,9.5,9.600000000000001,9.700000000000001,9.8,9.9,10.0],"y":[1.1464835179773751,1.6634539298784474,2.3840584404423515,3.3596322973215114,4.629504330019647,6.200510377447753,8.026246797750963,10.000000000000002,11.973753202249041,13.79948962255225,15.370495669980354,16.640367702678493,17.61594155955765,18.336546070121553,18.853516482022627,19.216685544064713,19.46806012846268,19.640275800758168,19.757431300314515,19.836748576936802,19.89027402201099,19.92631520201128,19.950547536867305,19.966823978396512,19.977749279342795,19.985079423323267,19.98999597785841,19.99329299739067,19.995503664595333,19.996985792838807,19.997979416121847,19.998645517007606,19.99909204262595,19.999391368861986,19.9995920182544,19.999726519818307,19.999816679925605,19.999877116507957,19.9999176285651,19.999944784700993,19.99996298804454,19.999975190129177,19.999983369439445,19.99998885219883,19.999992527404036,19.99999499096851,19.999996642345504,19.99999774929676,19.999998491308446,19.999998988693783,19.999999322101157,19.99999954559081,19.999999695400412,19.999999795820788,19.99999986313458,19.999999908256367,19.999999938502402,19.999999958776925,19.99999997236735,19.99999998147728,19.99999998758385,19.999999991677203,19.999999994421064,19.99999999626033,19.999999997493223,19.999999998319655,19.999999998873633,19.999999999244974,19.99999999949389,19.999999999660744,19.99999999977259,19.999999999847564,19.99999999989782,19.999999999931504,19.999999999954085,19.999999999969226,19.999999999979373,19.999999999986173,19.99999999999073,19.999999999993786,19.999999999995836,19.999999999997208,19.99999999999813,19.99999999999875,19.99999999999916,19.999999999999435,19.999999999999623,19.999999999999748,19.999999999999833,19.999999999999886,19.999999999999925,19.999999999999947,19.999999999999964,19.99999999999998,19.999999999999986,19.999999999999993,19.999999999999993,19.999999999999996,19.999999999999996,20.0,20.0],"marker":{},"line":{},"name":"vector [|20.0; 4.0; 0.7|]"},{"type":"scatter","mode":"lines","x":[0.0,0.1,0.2,0.30000000000000004,0.4,0.5,0.6000000000000001,0.7000000000000001,0.8,0.9,1.0,1.1,1.2000000000000002,1.3,1.4000000000000001,1.5,1.6,1.7000000000000002,1.8,1.9000000000000001,2.0,2.1,2.2,2.3000000000000003,2.4000000000000004,2.5,2.6,2.7,2.8000000000000003,2.9000000000000004,3.0,3.1,3.2,3.3000000000000003,3.4000000000000004,3.5,3.6,3.7,3.8000000000000003,3.9000000000000004,4.0,4.1000000000000005,4.2,4.3,4.4,4.5,4.6000000000000005,4.7,4.800000000000001,4.9,5.0,5.1000000000000005,5.2,5.300000000000001,5.4,5.5,5.6000000000000005,5.7,5.800000000000001,5.9,6.0,6.1000000000000005,6.2,6.300000000000001,6.4,6.5,6.6000000000000005,6.7,6.800000000000001,6.9,7.0,7.1000000000000005,7.2,7.300000000000001,7.4,7.5,7.6000000000000005,7.7,7.800000000000001,7.9,8.0,8.1,8.200000000000001,8.3,8.4,8.5,8.6,8.700000000000001,8.8,8.9,9.0,9.1,9.200000000000001,9.3,9.4,9.5,9.600000000000001,9.700000000000001,9.8,9.9,10.0],"y":[6.880772588407907E-08,1.2537585047267886E-07,2.2844969269140523E-07,4.162624747844818E-07,7.584796638975661E-07,1.3820399978492664E-06,2.51824087222428E-06,4.5885334038843705E-06,8.360850878784121E-06,1.5234456592443794E-05,2.7758966592951754E-05,5.0580057955361715E-05,9.216261903322076E-05,0.0001679303925766994,0.0003059863091988206,0.000557534056543341,0.0010158622442965517,0.0018509186397934758,0.003372251553499429,0.006143507474790751,0.011190432507550445,0.020377799256434372,0.037089347349521615,0.06744409741411773,0.12243856729739855,0.22161047539909565,0.3989549036529876,0.7113880976635018,1.2475904474088362,2.127765973507319,3.472128247514733,5.315155406613067,7.5,9.684844593386934,11.527871752485268,12.872234026492682,13.752409552591166,14.2886119023365,14.601045096347011,14.778389524600904,14.877561432702601,14.932555902585882,14.96291065265048,14.979622200743567,14.988809567492451,14.99385649252521,14.9966277484465,14.998149081360205,14.998984137755706,14.999442465943456,14.999694013690801,14.999832069607425,14.999907837380967,14.999949419942045,14.999972241033406,14.99998476554341,14.999991639149123,14.999995411466594,14.999997481759127,14.999998617960001,14.999999241520335,14.999999583737523,14.999999771550307,14.999999874624148,14.999999931192274,14.99999996223752,14.999999979275511,14.999999988626158,14.999999993757903,14.999999996574264,14.999999998119918,14.999999998968189,14.99999999943373,14.999999999689225,14.999999999829443,14.999999999906395,14.999999999948628,14.999999999971806,14.99999999998453,14.999999999991507,14.99999999999534,14.999999999997442,14.999999999998598,14.99999999999923,14.999999999999577,14.999999999999767,14.999999999999874,14.99999999999993,14.999999999999961,14.99999999999998,14.99999999999999,14.999999999999993,14.999999999999996,14.999999999999996,15.0,15.0,15.0,15.0,15.0,15.0,15.0],"marker":{},"line":{},"name":"vector [|15.0; 6.0; 3.2|]"}];
    var layout = {"width":600,"height":600,"template":{"layout":{"paper_bgcolor":"white","plot_bgcolor":"white","xaxis":{"ticks":"inside","mirror":"all","showline":true,"zeroline":true},"yaxis":{"ticks":"inside","mirror":"all","showline":true,"zeroline":true}},"data":{}},"title":{"text":"Verhulst"}};
    var config = {"responsive":true};
    Plotly.newPlot('f4084ece-17f3-4612-9bd9-b0d625f0bb10', data, layout, config);
};
renderPlotly_f4084ece17f346129bd9b0d625f0bb10();
</script></div>

<div><div id="6b992995-8013-42e9-88ad-24688e280d99"><!-- Plotly chart will be drawn inside this DIV --></div><script type="text/javascript">var renderPlotly_6b992995801342e988ad24688e280d99 = function() {
    var data = [{"type":"scatter","mode":"lines","x":[0.0,0.1,0.2,0.30000000000000004,0.4,0.5,0.6000000000000001,0.7000000000000001,0.8,0.9,1.0,1.1,1.2000000000000002,1.3,1.4000000000000001,1.5,1.6,1.7000000000000002,1.8,1.9000000000000001,2.0,2.1,2.2,2.3000000000000003,2.4000000000000004,2.5,2.6,2.7,2.8000000000000003,2.9000000000000004,3.0,3.1,3.2,3.3000000000000003,3.4000000000000004,3.5,3.6,3.7,3.8000000000000003,3.9000000000000004,4.0,4.1000000000000005,4.2,4.3,4.4,4.5,4.6000000000000005,4.7,4.800000000000001,4.9,5.0,5.1000000000000005,5.2,5.300000000000001,5.4,5.5,5.6000000000000005,5.7,5.800000000000001,5.9,6.0,6.1000000000000005,6.2,6.300000000000001,6.4,6.5,6.6000000000000005,6.7,6.800000000000001,6.9,7.0,7.1000000000000005,7.2,7.300000000000001,7.4,7.5,7.6000000000000005,7.7,7.800000000000001,7.9,8.0,8.1,8.200000000000001,8.3,8.4,8.5,8.6,8.700000000000001,8.8,8.9,9.0,9.1,9.200000000000001,9.3,9.4,9.5,9.600000000000001,9.700000000000001,9.8,9.9,10.0],"y":[10.474258731775668,10.521535630784177,10.573241758988688,10.629733560569965,10.691384203433469,10.758581800212436,10.831726964939223,10.911229610148562,10.99750489119685,11.09096821195613,11.192029220221176,11.301084743629978,11.418510649004878,11.544652650835348,11.679816148660755,11.824255238063564,11.978161114414183,12.141650169574415,12.314752165009823,12.497398944048825,12.689414213699951,12.89050497374996,13.100255188723876,13.31812227831834,13.543436937742047,13.775406687981455,14.013123398875479,14.255574831883411,14.501660026875221,14.750208125210602,15.0,15.2497918747894,15.498339973124779,15.74442516811659,15.986876601124521,16.224593312018545,16.456563062257956,16.681877721681662,16.899744811276125,17.10949502625004,17.31058578630005,17.502601055951175,17.68524783499018,17.85834983042559,18.02183888558582,18.175744761936436,18.320183851339245,18.455347349164654,18.58148935099512,18.69891525637002,18.807970779778824,18.90903178804387,19.00249510880315,19.08877038985144,19.16827303506078,19.241418199787567,19.308615796566535,19.370266439430033,19.426758241011314,19.47846436921582,19.525741268224333,19.568927450589136,19.60834277203236,19.64428810727364,19.677045353015494,19.706877692486437,19.73403006423134,19.758729785823306,19.781187290638695,19.801596942659224,19.820137900379084,19.83697500628559,19.85225968306727,19.866130821723353,19.878715650157257,19.89013057369407,19.900481981330955,19.90986701347152,19.9183742884684,19.92608458655718,19.93307149075715,19.93940198508416,19.945137011005492,19.95033198349943,19.95503726839059,19.95929862284104,19.96315760100564,19.966651926925866,19.969815836752915,19.972680392369888,19.975273768433652,19.977621514787238,19.979746796109502,19.981670610575073,19.983411989198256,19.98498817743263,19.98641480049571,19.987706013787225,19.988874639671398,19.989932291799143,19.990889488055995],"marker":{},"line":{},"name":"vector [|20.0; 3.0; 1.0; 10.0|]"},{"type":"scatter","mode":"lines","x":[0.0,0.1,0.2,0.30000000000000004,0.4,0.5,0.6000000000000001,0.7000000000000001,0.8,0.9,1.0,1.1,1.2000000000000002,1.3,1.4000000000000001,1.5,1.6,1.7000000000000002,1.8,1.9000000000000001,2.0,2.1,2.2,2.3000000000000003,2.4000000000000004,2.5,2.6,2.7,2.8000000000000003,2.9000000000000004,3.0,3.1,3.2,3.3000000000000003,3.4000000000000004,3.5,3.6,3.7,3.8000000000000003,3.9000000000000004,4.0,4.1000000000000005,4.2,4.3,4.4,4.5,4.6000000000000005,4.7,4.800000000000001,4.9,5.0,5.1000000000000005,5.2,5.300000000000001,5.4,5.5,5.6000000000000005,5.7,5.800000000000001,5.9,6.0,6.1000000000000005,6.2,6.300000000000001,6.4,6.5,6.6000000000000005,6.7,6.800000000000001,6.9,7.0,7.1000000000000005,7.2,7.300000000000001,7.4,7.5,7.6000000000000005,7.7,7.800000000000001,7.9,8.0,8.1,8.200000000000001,8.3,8.4,8.5,8.6,8.700000000000001,8.8,8.9,9.0,9.1,9.200000000000001,9.3,9.4,9.5,9.600000000000001,9.700000000000001,9.8,9.9,10.0],"y":[10.649691691286641,10.691384203433469,10.73554176934819,10.78228259451784,10.831726964939223,10.883996772070583,10.939214960192711,10.997504891196852,11.058989622359178,11.123791093413567,11.192029220221176,11.263820893580128,11.339278883240738,11.418510649004878,11.501617062898474,11.588691048809151,11.679816148660755,11.775065027120739,11.874497929957412,11.978161114414183,12.08608527326045,12.198283977406826,12.314752165009823,12.435464707695958,12.56037508675242,12.689414213699951,12.8224894304226,12.959483723828376,13.100255188723876,13.244636770105458,13.392436312341829,13.543436937742046,13.697397770822487,13.854055017324505,14.01312339887548,14.174297935376853,14.337256058045607,14.501660026875221,14.667159617488686,14.833395034305887,15.0,15.166604965694116,15.332840382511314,15.498339973124779,15.662743941954393,15.825702064623147,15.986876601124521,16.145944982675495,16.302602229177516,16.456563062257956,16.60756368765817,16.75536322989454,16.899744811276125,17.040516276171626,17.1775105695774,17.31058578630005,17.43962491324758,17.564535292304043,17.68524783499018,17.801716022593176,17.913914726739552,18.02183888558582,18.125502070042586,18.22493497287926,18.320183851339245,18.411308951190847,18.49838293710153,18.58148935099512,18.660721116759262,18.736179106419872,18.807970779778824,18.876208906586434,18.941010377640822,19.00249510880315,19.06078503980729,19.11600322792942,19.16827303506078,19.217717405482162,19.26445823065181,19.308615796566535,19.35030830871336,19.38965148927008,19.426758241011314,19.461738372307703,19.494698377645868,19.525741268224333,19.55496644737713,19.58246962582418,19.60834277203236,19.632674093279704,19.655548043337888,19.677045353015494,19.697243080130495,19.716214675798643,19.73403006423134,19.750755733528862,19.766454835230228,19.781187290638695,19.795009902180183,19.80797646827191,19.820137900379084],"marker":{},"line":{},"name":"vector [|20.0; 4.0; 1.5; 10.0|]"},{"type":"scatter","mode":"lines","x":[0.0,0.1,0.2,0.30000000000000004,0.4,0.5,0.6000000000000001,0.7000000000000001,0.8,0.9,1.0,1.1,1.2000000000000002,1.3,1.4000000000000001,1.5,1.6,1.7000000000000002,1.8,1.9000000000000001,2.0,2.1,2.2,2.3000000000000003,2.4000000000000004,2.5,2.6,2.7,2.8000000000000003,2.9000000000000004,3.0,3.1,3.2,3.3000000000000003,3.4000000000000004,3.5,3.6,3.7,3.8000000000000003,3.9000000000000004,4.0,4.1000000000000005,4.2,4.3,4.4,4.5,4.6000000000000005,4.7,4.800000000000001,4.9,5.0,5.1000000000000005,5.2,5.300000000000001,5.4,5.5,5.6000000000000005,5.7,5.800000000000001,5.9,6.0,6.1000000000000005,6.2,6.300000000000001,6.4,6.5,6.6000000000000005,6.7,6.800000000000001,6.9,7.0,7.1000000000000005,7.2,7.300000000000001,7.4,7.5,7.6000000000000005,7.7,7.800000000000001,7.9,8.0,8.1,8.200000000000001,8.3,8.4,8.5,8.6,8.700000000000001,8.8,8.9,9.0,9.1,9.200000000000001,9.3,9.4,9.5,9.600000000000001,9.700000000000001,9.8,9.9,10.0],"y":[10.012363115783174,10.013659803815056,10.015092081623543,10.016674036537067,10.01842119949718,10.02035068857948,10.022481365804706,10.024834008250284,10.027431494497252,10.03029900745792,10.033464254621425,10.03695770672141,10.0408128557658,10.045066493264239,10.049759009334522,10.054934713152965,10.060642174921371,10.066934589138324,10.073870158466365,10.081512496857204,10.089931049810458,10.099201528670388,10.109406354680653,10.120635107088345,10.13298496788433,10.146561153756782,10.161477323492253,10.17785594636318,10.195828613983823,10.21553627470543,10.237129365887833,10.26076781539209,10.286620879494343,10.314866780284982,10.345692101716734,10.379290900106218,10.415863482469613,10.45561480507428,10.498752445598425,10.545484105978066,10.596014610110588,10.65054237181499,10.70925532450244,10.772326325417673,10.839908074330378,10.912127619031782,10.989080557207092,11.070825084787208,11.157376082504912,11.248699472024413,11.344707106849976,11.445252486874981,11.550127594361939,11.65906113915917,11.771718468871024,11.887703343990728,12.006561699437741,12.127787415941706,12.25083001343761,12.3751040626053,12.5,12.624895937394701,12.74916998656239,12.872212584058296,12.99343830056226,13.112296656009272,13.228281531128978,13.340938860840831,13.449872405638063,13.55474751312502,13.655292893150024,13.751300527975587,13.84262391749509,13.929174915212794,14.01091944279291,14.087872380968218,14.160091925669622,14.227673674582327,14.29074467549756,14.34945762818501,14.403985389889412,14.454515894021934,14.501247554401576,14.54438519492572,14.58413651753039,14.620709099893784,14.654307898283266,14.685133219715016,14.713379120505657,14.73923218460791,14.762870634112167,14.784463725294568,14.80417138601618,14.82214405363682,14.838522676507747,14.853438846243218,14.86701503211567,14.879364892911653,14.890593645319347,14.900798471329612,14.910068950189542],"marker":{},"line":{},"name":"vector [|15.0; 6.0; 1.0; 10.0|]"}];
    var layout = {"width":600,"height":600,"template":{"layout":{"paper_bgcolor":"white","plot_bgcolor":"white","xaxis":{"ticks":"inside","mirror":"all","showline":true,"zeroline":true},"yaxis":{"ticks":"inside","mirror":"all","showline":true,"zeroline":true}},"data":{}},"title":{"text":"Verhulst 4 Param"}};
    var config = {"responsive":true};
    Plotly.newPlot('6b992995-8013-42e9-88ad-24688e280d99', data, layout, config);
};
renderPlotly_6b992995801342e988ad24688e280d99();
</script></div>

<div><div id="facba0e4-73e2-4750-9c28-5932b2350e28"><!-- Plotly chart will be drawn inside this DIV --></div><script type="text/javascript">var renderPlotly_facba0e473e247509c285932b2350e28 = function() {
    var data = [{"type":"scatter","mode":"lines","x":[0.0,0.1,0.2,0.30000000000000004,0.4,0.5,0.6000000000000001,0.7000000000000001,0.8,0.9,1.0,1.1,1.2000000000000002,1.3,1.4000000000000001,1.5,1.6,1.7000000000000002,1.8,1.9000000000000001,2.0,2.1,2.2,2.3000000000000003,2.4000000000000004,2.5,2.6,2.7,2.8000000000000003,2.9000000000000004,3.0,3.1,3.2,3.3000000000000003,3.4000000000000004,3.5,3.6,3.7,3.8000000000000003,3.9000000000000004,4.0,4.1000000000000005,4.2,4.3,4.4,4.5,4.6000000000000005,4.7,4.800000000000001,4.9,5.0,5.1000000000000005,5.2,5.300000000000001,5.4,5.5,5.6000000000000005,5.7,5.800000000000001,5.9,6.0,6.1000000000000005,6.2,6.300000000000001,6.4,6.5,6.6000000000000005,6.7,6.800000000000001,6.9,7.0,7.1000000000000005,7.2,7.300000000000001,7.4,7.5,7.6000000000000005,7.7,7.800000000000001,7.9,8.0,8.1,8.200000000000001,8.3,8.4,8.5,8.6,8.700000000000001,8.8,8.9,9.0,9.1,9.200000000000001,9.3,9.4,9.5,9.600000000000001,9.700000000000001,9.8,9.9,10.0],"y":[8.0,8.000095999232006,8.000767950851147,8.002591440248906,8.006140855881789,8.011988011988011,8.020700230002555,8.032837892822098,8.048951494677802,8.069578219821999,8.095238095238095,8.126429775747837,8.163626033709994,8.207269039364137,8.257765531062123,8.315481986368061,8.380739914482245,8.453811396857898,8.53491500550324,8.624212226696699,8.721804511278195,8.827731061142103,8.941967445152157,9.064425116828392,9.194951881519046,9.333333333333334,9.47929525305802,9.632506928941202,9.792585333986608,9.959100067608725,10.131578947368423,10.30951411903793,10.49236854114903,10.679582693849147,10.870581361378909,11.064780342516753,11.26159295334856,11.460436200918858,11.660736523750224,11.861935014838407,12.063492063492063,12.264891373291187,12.465643333601221,12.66528774076193,12.863395881703651,13.059572006940428,13.253454231433507,13.444714910651463,13.63306054534959,13.81823127233164,14.0,14.178171247152157,14.352579741574049,14.523088831847875,14.689588761753711,14.851994851994853,15.010245628975172,15.164300935224201,15.314140050982157,15.459759851564417,15.60117302052786,15.738406334432826,15.871499031178441,16.00050127050743,16.125472692331538,16.24648107600876,16.36360110158644,16.47691321228007,16.586502576052723,16.692458143057802,16.794871794871796,16.89383758083788,16.989451036431657,17.081808578315503,17.171006970638825,17.257142857142856,17.34031235371718,17.420610696211565,17.498131938514092,17.572968696150244,17.645211930926216,17.714950772422807,17.782272372436307,17.847261788753237,17.910001894931614,17.97057331303907,18.029054366564353,18.085521050972364,18.140047019611345,18.192703582904905,18.243559718969557,18.292682093991267,18.34013509087207,18.38598084482067,18.430279284709666,18.473088179157653,18.514463186417046,18.554457907259803,18.59312394015337,18.630510938109136,18.666666666666668],"marker":{},"line":{},"name":"vector [|8.0; 20.0; 0.2; 3.0|]"},{"type":"scatter","mode":"lines","x":[0.0,0.1,0.2,0.30000000000000004,0.4,0.5,0.6000000000000001,0.7000000000000001,0.8,0.9,1.0,1.1,1.2000000000000002,1.3,1.4000000000000001,1.5,1.6,1.7000000000000002,1.8,1.9000000000000001,2.0,2.1,2.2,2.3000000000000003,2.4000000000000004,2.5,2.6,2.7,2.8000000000000003,2.9000000000000004,3.0,3.1,3.2,3.3000000000000003,3.4000000000000004,3.5,3.6,3.7,3.8000000000000003,3.9000000000000004,4.0,4.1000000000000005,4.2,4.3,4.4,4.5,4.6000000000000005,4.7,4.800000000000001,4.9,5.0,5.1000000000000005,5.2,5.300000000000001,5.4,5.5,5.6000000000000005,5.7,5.800000000000001,5.9,6.0,6.1000000000000005,6.2,6.300000000000001,6.4,6.5,6.6000000000000005,6.7,6.800000000000001,6.9,7.0,7.1000000000000005,7.2,7.300000000000001,7.4,7.5,7.6000000000000005,7.7,7.800000000000001,7.9,8.0,8.1,8.200000000000001,8.3,8.4,8.5,8.6,8.700000000000001,8.8,8.9,9.0,9.1,9.200000000000001,9.3,9.4,9.5,9.600000000000001,9.700000000000001,9.8,9.9,10.0],"y":[10.0,10.000003906248473,10.000062499609378,10.000316396239025,10.00099990001,10.002440810349036,10.00505993840618,10.009370118104075,10.015974440894569,10.025563390077227,10.038910505836576,10.056866180566532,10.080349171709155,10.110335433470567,10.14784391721726,10.193919080679914,10.249609984399376,10.315946043174993,10.393909738774383,10.484406879327784,10.588235294117647,10.706053154841149,10.838348383254793,10.985410802928756,11.14730878186969,11.323872061004025,11.514682251366933,11.719072109090876,11.9361341827272,12.16473881493511,12.403560830860535,12.651113628997788,12.905788876276958,13.165899654584281,13.4297247465311,13.695551793135294,13.961717287603406,14.226641748010504,14.488858883787257,14.7470380784337,15.0,15.246725578799358,15.486358930379907,15.718205038100448,15.941723144352908,16.156516843389323,16.362321839028194,16.55899224711382,16.74648620510151,16.924851416868705,17.094211123723042,17.2547508627428,17.4067062576178,17.55035198858311,17.685992007994898,17.813951006030848,17.934567085261072,18.048185571196587,18.155153865932725,18.255817241260196,18.350515463917525,18.43958014709517,18.523332727284473,18.602082972819087,18.676127938995975,18.745751293750192,18.811222946963454,18.872798925249,18.93072144224292,18.985219121912486,19.036507339104254,19.08478864748958,19.130253270247703,19.173079633303455,19.21343492476751,19.251475667476836,19.287348294268142,19.32118971790156,19.353127889446323,19.38328234049941,19.41176470588235,19.438679224494127,19.46412321682814,19.488187538322208,19.510957008230594,19.532510814111415,19.552922892331818,19.572262285224497,19.590593475697627,19.607976700217684,19.6244682411618,19.640120699581182,19.65498324943676,19.669101874368554,19.682519588045356,19.695276639115587,19.707410701746173,19.718957052696464,19.72994873583096,19.74041671492871,19.750390015600622],"marker":{},"line":{},"name":"vector [|10.0; 20.0; 0.25; 4.0|]"},{"type":"scatter","mode":"lines","x":[0.0,0.1,0.2,0.30000000000000004,0.4,0.5,0.6000000000000001,0.7000000000000001,0.8,0.9,1.0,1.1,1.2000000000000002,1.3,1.4000000000000001,1.5,1.6,1.7000000000000002,1.8,1.9000000000000001,2.0,2.1,2.2,2.3000000000000003,2.4000000000000004,2.5,2.6,2.7,2.8000000000000003,2.9000000000000004,3.0,3.1,3.2,3.3000000000000003,3.4000000000000004,3.5,3.6,3.7,3.8000000000000003,3.9000000000000004,4.0,4.1000000000000005,4.2,4.3,4.4,4.5,4.6000000000000005,4.7,4.800000000000001,4.9,5.0,5.1000000000000005,5.2,5.300000000000001,5.4,5.5,5.6000000000000005,5.7,5.800000000000001,5.9,6.0,6.1000000000000005,6.2,6.300000000000001,6.4,6.5,6.6000000000000005,6.7,6.800000000000001,6.9,7.0,7.1000000000000005,7.2,7.300000000000001,7.4,7.5,7.6000000000000005,7.7,7.800000000000001,7.9,8.0,8.1,8.200000000000001,8.3,8.4,8.5,8.6,8.700000000000001,8.8,8.9,9.0,9.1,9.200000000000001,9.3,9.4,9.5,9.600000000000001,9.700000000000001,9.8,9.9,10.0],"y":[10.0,10.000134996355097,10.001079766770378,10.003642344730691,10.008625095834397,10.016818238445246,10.028990924925832,10.045880104353582,10.068177514045832,10.096515289555676,10.13145082765336,10.173451667427653,10.222881252293018,10.279986481881284,10.344887942142542,10.417573605223966,10.497896617299602,10.585577552132122,10.680211220392167,10.781277817199392,10.888157894736842,11.000150394345173,11.11649278910381,11.236382290221595,11.358997063895233,11.483516483516484,11.609139589516003,11.735101123712894,11.860684722333843,11.985233068616699,12.108155002891845,12.228929751706563,12.347108560562328,12.462314095474849,12.574238019646732,12.682637158913156,12.787328650730712,12.888184434145355,12.98512538967675,13.078115384571033,13.167155425219942,13.252278068152068,13.333542196044762,13.411028226812373,13.484833792327683,13.55506989849366,13.621857559607417,13.685324886458083,13.745604598513191,13.802831925024716,13.857142857142858,13.90867271247433,13.957554974380871,14.00392037020694,14.04789615517617,14.089605571613902,14.129167456221506,14.166695971191679,14.202300437905151,14.236085254718969,14.26814988290398,14.298588887094835,14.327492018675278,14.354944332340937,14.381026327673768,14.405814108943982,14.429379557545474,14.451790512489254,14.473110955245623,14.493401195958192,14.512718058668746,14.531115063706928,14.54864260582706,14.565348127028575,14.581276283287698,14.596469104665825,14.610966148452828,14.624804645158617,14.638019637289885,14.650644110946207,14.662709120345387,14.674243905445515,14.68527600287431,14.695831350407213,14.70593438525701,14.715608136451216,14.724874311580502,14.7337533782037,14.74226464019319,14.750426309299769,14.75825557220906,14.765768653352925,14.77298087372942,14.779906705974156,14.786559825914779,14.792953160828809,14.799098934613614,14.80500871006593,14.810693428457109,14.81616344657947,14.821428571428571],"marker":{},"line":{},"name":"vector [|10.0; 15.0; 0.3; 3.0|]"}];
    var layout = {"width":600,"height":600,"template":{"layout":{"paper_bgcolor":"white","plot_bgcolor":"white","xaxis":{"ticks":"inside","mirror":"all","showline":true,"zeroline":true},"yaxis":{"ticks":"inside","mirror":"all","showline":true,"zeroline":true}},"data":{}},"title":{"text":"MMF"}};
    var config = {"responsive":true};
    Plotly.newPlot('facba0e4-73e2-4750-9c28-5932b2350e28', data, layout, config);
};
renderPlotly_facba0e473e247509c285932b2350e28();
</script></div>

<div><div id="b3b2c685-cef6-4f35-9c78-4bf5ec2cf34b"><!-- Plotly chart will be drawn inside this DIV --></div><script type="text/javascript">var renderPlotly_b3b2c685cef64f359c784bf5ec2cf34b = function() {
    var data = [{"type":"scatter","mode":"lines","x":[0.0,0.1,0.2,0.30000000000000004,0.4,0.5,0.6000000000000001,0.7000000000000001,0.8,0.9,1.0,1.1,1.2000000000000002,1.3,1.4000000000000001,1.5,1.6,1.7000000000000002,1.8,1.9000000000000001,2.0,2.1,2.2,2.3000000000000003,2.4000000000000004,2.5,2.6,2.7,2.8000000000000003,2.9000000000000004,3.0,3.1,3.2,3.3000000000000003,3.4000000000000004,3.5,3.6,3.7,3.8000000000000003,3.9000000000000004,4.0,4.1000000000000005,4.2,4.3,4.4,4.5,4.6000000000000005,4.7,4.800000000000001,4.9,5.0,5.1000000000000005,5.2,5.300000000000001,5.4,5.5,5.6000000000000005,5.7,5.800000000000001,5.9,6.0,6.1000000000000005,6.2,6.300000000000001,6.4,6.5,6.6000000000000005,6.7,6.800000000000001,6.9,7.0,7.1000000000000005,7.2,7.300000000000001,7.4,7.5,7.6000000000000005,7.7,7.800000000000001,7.9,8.0,8.1,8.200000000000001,8.3,8.4,8.5,8.6,8.700000000000001,8.8,8.9,9.0,9.1,9.200000000000001,9.3,9.4,9.5,9.600000000000001,9.700000000000001,9.8,9.9,10.0],"y":[-16.442376007810182,-15.720768615001468,-15.01345000592202,-14.320137243697172,-13.640552993977728,-12.974425414002564,-12.321488043857869,-11.681479699889636,-11.054144370226723,-10.439231112372678,-9.836493952825407,-9.24569178868449,-8.666588291206804,-8.098951811271874,-7.542555286719144,-6.997176151520064,-6.4625962467487374,-5.938601733315436,-5.424983006428095,-4.9215346117476155,-4.428055163203397,-3.944347262436203,-3.4702174198362057,-3.0054759771445427,-2.5499370315875103,-2.1034183615129542,-1.6657413534991727,-1.2367309309071928,-0.8162154838477642,-0.4040268005351155,0.0,0.39602653386489495,0.7842112169535365,1.1647093283150278,1.537673072267285,1.9032516392808096,2.2615912656568504,2.612835292023885,2.9571242206757753,3.29459577177456,3.6253849384403636,3.9496240407504324,4.267442778668933,4.578968283928672,4.884325170885493,5.183635586365643,5.477019258526184,5.764593544747807,6.046473478579381,6.322771815752883,6.5935990792872134,6.859063603698868,7.119271578337174,7.374327089861483,7.624332163877183,7.8693868057473315,8.109589040596115,8.345034952520207,8.575818723023705,8.80203266869196,9.023767278119472,9.24111124810651,9.454151519139028,9.662973310166016,9.86766015268821,10.068293924171812,10.26495488080057,10.457721689579314,10.646671459801818,10.83187977389553,11.013420717655569,11.191366909880017,11.365789531418407,11.536758353645027,11.704341766368373,11.868606805188017,12.02961917830972,12.187443292829578,12.342142280497761,12.49377802297201,12.642411176571153,12.788101196538433,12.930906360824398,13.070883793398853,13.208089487101217,13.34257832603841,13.474404107539208,13.603619563673925,13.730276382347895,13.854425227977377,13.97611576175596,14.095396661519715,14.21231564121899,14.326919470004595,14.439253990936118,14.54936413931975,14.657293960682996,14.763086628393483,14.866784460928884,14.96842893880487,15.068060721167871],"marker":{},"line":{},"name":"vector [|20.0; 0.2; 3.0|]"},{"type":"scatter","mode":"lines","x":[0.0,0.1,0.2,0.30000000000000004,0.4,0.5,0.6000000000000001,0.7000000000000001,0.8,0.9,1.0,1.1,1.2000000000000002,1.3,1.4000000000000001,1.5,1.6,1.7000000000000002,1.8,1.9000000000000001,2.0,2.1,2.2,2.3000000000000003,2.4000000000000004,2.5,2.6,2.7,2.8000000000000003,2.9000000000000004,3.0,3.1,3.2,3.3000000000000003,3.4000000000000004,3.5,3.6,3.7,3.8000000000000003,3.9000000000000004,4.0,4.1000000000000005,4.2,4.3,4.4,4.5,4.6000000000000005,4.7,4.800000000000001,4.9,5.0,5.1000000000000005,5.2,5.300000000000001,5.4,5.5,5.6000000000000005,5.7,5.800000000000001,5.9,6.0,6.1000000000000005,6.2,6.300000000000001,6.4,6.5,6.6000000000000005,6.7,6.800000000000001,6.9,7.0,7.1000000000000005,7.2,7.300000000000001,7.4,7.5,7.6000000000000005,7.7,7.800000000000001,7.9,8.0,8.1,8.200000000000001,8.3,8.4,8.5,8.6,8.700000000000001,8.8,8.9,9.0,9.1,9.200000000000001,9.3,9.4,9.5,9.600000000000001,9.700000000000001,9.8,9.9,10.0],"y":[-34.3656365691809,-33.02334421965213,-31.71419318631692,-30.43736520716296,-29.192062223138997,-27.97750587934196,-26.792937038519817,-25.63761530658607,-24.510818569849356,-23.41184254366885,-22.340000332253496,-21.294621999329728,-20.275054149409534,-19.28065951939695,-18.31081658027792,-17.364919148644447,-16.44237600781018,-15.542610538280766,-14.665060357347905,-13.809176967581823,-12.974425414002564,-12.160283949715659,-11.366243709803374,-10.59180839326757,-9.836493952825403,-9.099828292364025,-8.381350971865142,-7.680612919615029,-6.997176151520059,-6.330613497352431,-5.680508333754828,-5.046454323837288,-4.428055163203397,-3.8249243322471616,-3.2366848545656612,-2.6629690613365264,-2.10341836151295,-1.5576830176926304,-1.025421927520478,-0.5063024104885772,0.0,0.4938017594333499,0.9754115099857197,1.4451302734289406,1.9032516392808096,2.350061948308091,2.785840471498846,3.2108595846158527,3.625384938440366,4.0296756248124606,4.423984338571902,4.808557535500633,5.183635586365643,5.549452927158558,5.906238205625733,6.254214424180555,6.593599079287216,6.924604297403054,7.247436967564536,7.5622988706996,7.8693868057473315,8.1688927126637,8.461003792390267,8.74590262386089,9.023767278119472,9.294771429620194,9.559084464779682,9.816871587849016,10.068293924171812,10.313508620892753,10.552668945179706,10.785924380020685,11.013420717655569,11.235300150701018,11.451701361025467,11.66275960642983,11.86860680518802,12.069371618500142,12.265179530909977,12.456152928736861,12.642411176571153,12.824070691880966,13.001245017776895,13.174044893980128,13.34257832603841,13.506950652833005,13.667264612418935,13.823620406239606,13.97611576175596,14.124845993529345,14.269904062796197,14.411380635571852,14.54936413931975,14.683940818221469,14.81519478708217,14.94320808390507,15.068060721167873,15.189830735833159,15.308594238124048,15.424425459095552,15.537396797031404],"marker":{},"line":{},"name":"vector [|20.0; 0.25; 4.0|]"},{"type":"scatter","mode":"lines","x":[0.0,0.1,0.2,0.30000000000000004,0.4,0.5,0.6000000000000001,0.7000000000000001,0.8,0.9,1.0,1.1,1.2000000000000002,1.3,1.4000000000000001,1.5,1.6,1.7000000000000002,1.8,1.9000000000000001,2.0,2.1,2.2,2.3000000000000003,2.4000000000000004,2.5,2.6,2.7,2.8000000000000003,2.9000000000000004,3.0,3.1,3.2,3.3000000000000003,3.4000000000000004,3.5,3.6,3.7,3.8000000000000003,3.9000000000000004,4.0,4.1000000000000005,4.2,4.3,4.4,4.5,4.6000000000000005,4.7,4.800000000000001,4.9,5.0,5.1000000000000005,5.2,5.300000000000001,5.4,5.5,5.6000000000000005,5.7,5.800000000000001,5.9,6.0,6.1000000000000005,6.2,6.300000000000001,6.4,6.5,6.6000000000000005,6.7,6.800000000000001,6.9,7.0,7.1000000000000005,7.2,7.300000000000001,7.4,7.5,7.6000000000000005,7.7,7.800000000000001,7.9,8.0,8.1,8.200000000000001,8.3,8.4,8.5,8.6,8.700000000000001,8.8,8.9,9.0,9.1,9.200000000000001,9.3,9.4,9.5,9.600000000000001,9.700000000000001,9.8,9.9,10.0],"y":[-21.894046667354242,-20.803662802864146,-19.745504651716374,-18.718619800147074,-17.722083982473016,-16.75500024919012,-15.816498159658314,-14.905732998646235,-14.021885016030476,-13.164158688965147,-12.331782005857633,-11.524005771506026,-10.740102932772874,-9.979367924188296,-9.241116032893398,-8.524682782352532,-7.829423334279505,-7.154711908239637,-6.499941218405102,-5.864521926956704,-5.247882113640047,-4.64946676099871,-4.068737254821068,-3.505170899351148,-2.9582604468271487,-2.427513640924246,-1.912452773690636,-1.412614255578153,-0.9275481981803946,-0.45681800930275074,0.0,0.4433169967723777,0.8735319962362709,1.2910322209315788,1.6961934492426396,2.089380353624133,2.47094682883092,2.8412363104471936,3.2005820840016996,3.549307584947203,3.8877266897742317,4.216143998521109,4.534855108934535,4.844146882527529,5.144297702774149,5.4355777256734,5.718249122907889,5.992566317816012,6.258776214390157,6.517118419506944,6.767825458589602,7.011122984896542,7.247229982624512,7.476358964009168,7.6987161606004255,7.91450170888478,8.123909830421647,8.327129006655884,8.524342148563806,8.715726761285415,8.901455103891012,9.081694344425985,9.256606710373319,9.426349634669316,9.591075897403824,9.750933763332672,9.906067115325913,10.056615583872166,10.202714672755443,10.344495881017725,10.482086821316967,10.61561133478711,10.745189602503443,10.870938253653716,10.992970470512246,11.111396090311626,11.226321704103652,11.337850752698444,11.446083619768174,11.551117722199143,11.653047597773552,11.751964990259895,11.84795893198853,11.941115823986799,12.03151951374578,12.119251370688689,12.204390359408851,12.287013110743171,12.367193990745047,12.445005167618858,12.520516676676202,12.593796483372408,12.66491054448004,12.733922867454437,12.800895568044748,12.865888926202295,12.928961440336609,12.990169879967926,13.04956933682361,13.107213274424417,13.163153576205271],"marker":{},"line":{},"name":"vector [|15.0; 0.3; 3.0|]"}];
    var layout = {"width":600,"height":600,"template":{"layout":{"paper_bgcolor":"white","plot_bgcolor":"white","xaxis":{"ticks":"inside","mirror":"all","showline":true,"zeroline":true},"yaxis":{"ticks":"inside","mirror":"all","showline":true,"zeroline":true}},"data":{}},"title":{"text":"vonBertalanffy"}};
    var config = {"responsive":true};
    Plotly.newPlot('b3b2c685-cef6-4f35-9c78-4bf5ec2cf34b', data, layout, config);
};
renderPlotly_b3b2c685cef64f359c784bf5ec2cf34b();
</script></div>

